# Proyecto NLP - Diario El Peruano
## Extracción de Entidades y Análisis Temporal

**Autor:** [Nombre]  
**Curso:** Procesamiento de Lenguaje Natural  
**Fecha:** Noviembre 2025

---

## Contenido del Notebook

1. **Instalación de dependencias**
2. **Scraping de boletines**
3. **Extracción de texto**
4. **NER (Named Entity Recognition)**
5. **Análisis temporal**
6. **Visualizaciones**
7. **Conclusiones**

## PASO 1: Instalación de Dependencias

Ejecuta esta celda solo la primera vez:

In [1]:
# Instalar dependencias (ejecutar solo una vez)
!pip install requests beautifulsoup4 pandas pdfplumber spacy tqdm matplotlib seaborn
!python -m spacy download es_core_news_md


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     - -------------------------------------- 1.8/42.3 MB 13.2 MB/s eta 0:00:04
     --- ------------------------------------ 3.4/42.3 MB 9.9 MB/s eta 0:00:04
     ---- ----------------------------------- 4.7/42.3 MB 8.3 MB/s eta 0:00:05
     ----- ---------------------------------- 5.5/42.3 MB 7.0 MB/s eta 0:00:06
     ------ --------------------------------- 6.6/42.3 MB 6.6 MB/s eta 0:00:06
     ------ --------------------------------- 7.3/42.3 MB 6.2 MB/s eta 0:00:06
     ------- -------------------------------- 8.4/42.3 MB 6.0 MB/s eta 0:00:06
     -------- ------------------------------- 9.2/42.3 MB 5.7 MB/s eta 0:00:06
     --------- ------------------------------ 9.7/42.3 MB 5.4 MB/s eta 0:00:07
     --------- ------------------------------ 10.2/42.3 MB 5.1 MB/s eta 0:00:07
     ---------- ----------------------------- 11.3/42.3 MB 5.0 MB/s eta 0:00:07
     ----------- ---------------------------- 12.3/42.3 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## PASO 2: Importar Librerías

In [2]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from datetime import datetime, timedelta
import time
import pandas as pd
import pdfplumber
import json
import re
from collections import Counter
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print(" Librerías importadas correctamente")

 Librerías importadas correctamente


## PASO 3: Configuración

**IMPORTANTE**: Ajusta estas fechas según tus necesidades:

In [3]:
"""
# =========================
# CONFIGURACIÓN DEL PROYECTO  -  EN PAUSA
# =========================

# Fechas para descargar (MODIFICAR SEGÚN NECESITES)
# IMPORTANTE: Los fines de semana y feriados NO tienen publicación

# Opción 1: Últimos 7 días (PRUEBA)
#FECHA_FIN = datetime.now()
#FECHA_INICIO = FECHA_FIN - timedelta(days=7)

# Opción 2: Último mes (DESCOMENTAR SI QUIERES)
# FECHA_FIN = datetime.now()
# FECHA_INICIO = FECHA_FIN - timedelta(days=30)

# Opción 3: Fechas específicas (DESCOMENTAR SI QUIERES)
FECHA_INICIO = datetime(2025, 10, 1)
FECHA_FIN = datetime(2025, 10, 5)

# Carpetas
CARPETA_BOLETINES = Path("boletines")
CARPETA_TEXTOS = Path("textos")
CARPETA_GRAFICOS = Path("graficos")

# Crear carpetas
CARPETA_BOLETINES.mkdir(exist_ok=True)
CARPETA_TEXTOS.mkdir(exist_ok=True)
CARPETA_GRAFICOS.mkdir(exist_ok=True)

print(f" Período: {FECHA_INICIO.date()} a {FECHA_FIN.date()}")
print(f" Carpetas creadas: {CARPETA_BOLETINES}, {CARPETA_TEXTOS}, {CARPETA_GRAFICOS}")
"""

'\n# =========================\n# CONFIGURACIÓN DEL PROYECTO  -  EN PAUSA\n# =========================\n\n# Fechas para descargar (MODIFICAR SEGÚN NECESITES)\n# IMPORTANTE: Los fines de semana y feriados NO tienen publicación\n\n# Opción 1: Últimos 7 días (PRUEBA)\n#FECHA_FIN = datetime.now()\n#FECHA_INICIO = FECHA_FIN - timedelta(days=7)\n\n# Opción 2: Último mes (DESCOMENTAR SI QUIERES)\n# FECHA_FIN = datetime.now()\n# FECHA_INICIO = FECHA_FIN - timedelta(days=30)\n\n# Opción 3: Fechas específicas (DESCOMENTAR SI QUIERES)\nFECHA_INICIO = datetime(2025, 10, 1)\nFECHA_FIN = datetime(2025, 10, 5)\n\n# Carpetas\nCARPETA_BOLETINES = Path("boletines")\nCARPETA_TEXTOS = Path("textos")\nCARPETA_GRAFICOS = Path("graficos")\n\n# Crear carpetas\nCARPETA_BOLETINES.mkdir(exist_ok=True)\nCARPETA_TEXTOS.mkdir(exist_ok=True)\nCARPETA_GRAFICOS.mkdir(exist_ok=True)\n\nprint(f" Período: {FECHA_INICIO.date()} a {FECHA_FIN.date()}")\nprint(f" Carpetas creadas: {CARPETA_BOLETINES}, {CARPETA_TEXTOS}, {CARP

## PASO 4: Descargar Boletines del Diario El Peru
**NOTA:** La URL puede necesitar ajustes. Verifica en https://www.elperuano.pe/

In [4]:
# =========================================================================
# SCRAPER MULTI-MES - DIARIO EL PERUANO
# Descarga boletines según solicitud de meses
# =========================================================================

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from pathlib import Path
import requests
import re
import logging
from tqdm import tqdm
import random

# =========================================================================
# CONFIGURACIÓN
# =========================================================================

logging.basicConfig(
    filename='scraper_multimes.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

console = logging.StreamHandler()
console.setLevel(logging.INFO)
logging.getLogger('').addHandler(console)


# =========================================================================
# FUNCIÓN DE DESCARGA MEJORADA
# =========================================================================

def descargar_pdf_mejorado(url, nombre_archivo, max_reintentos=3):
    """
    Descarga PDF con:
    - Verificación más permisiva
    - Headers completos
    - Timeouts más largos
    - Pausa aleatoria entre reintentos
    """
    
    for intento in range(1, max_reintentos + 1):
        try:
            # Headers COMPLETOS para parecer navegador real
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Upgrade-Insecure-Requests': '1',
                'Referer': 'https://diariooficial.elperuano.pe/BoletinOficial',
            }
            
            # Timeout MÁS LARGO
            response = requests.get(
                url,
                timeout=120,  # 2 minutos
                headers=headers,
                stream=True,
                allow_redirects=True
            )
            
            if response.status_code == 200:
                # VERIFICACIÓN MÁS PERMISIVA
                content_type = response.headers.get('content-type', '').lower()
                
                # Aceptar tanto application/pdf COMO application/octet-stream
                es_pdf = (
                    'pdf' in content_type or
                    'octet-stream' in content_type or
                    'application/download' in content_type
                )
                
                if not es_pdf:
                    # Verificar por contenido (primeros bytes)
                    primeros_bytes = response.content[:4]
                    es_pdf_por_bytes = primeros_bytes == b'%PDF'
                    
                    if not es_pdf_por_bytes:
                        return {
                            'exito': False,
                            'tamaño_mb': 0,
                            'error': f'No es PDF (content-type: {content_type}, bytes: {primeros_bytes})'
                        }
                
                # Guardar
                with open(nombre_archivo, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                
                tamaño_mb = nombre_archivo.stat().st_size / (1024 * 1024)
                
                # Verificar tamaño
                if tamaño_mb < 0.05:  # Más permisivo: 50 KB mínimo
                    nombre_archivo.unlink()
                    return {
                        'exito': False,
                        'tamaño_mb': tamaño_mb,
                        'error': f'Archivo muy pequeño ({tamaño_mb:.2f} MB)'
                    }
                
                # Verificar que sea PDF válido leyendo
                try:
                    with open(nombre_archivo, 'rb') as f:
                        primeros = f.read(4)
                        if primeros != b'%PDF':
                            nombre_archivo.unlink()
                            return {
                                'exito': False,
                                'tamaño_mb': tamaño_mb,
                                'error': f'Archivo corrupto (no empieza con %PDF)'
                            }
                except:
                    pass  # Si no se puede leer, asumir que está bien
                
                logging.info(f" Descargado: {nombre_archivo.name} ({tamaño_mb:.2f} MB)")
                
                return {
                    'exito': True,
                    'tamaño_mb': tamaño_mb,
                    'error': None
                }
            
            else:
                error_msg = f'HTTP {response.status_code}'
                
                if intento < max_reintentos:
                    # Pausa ALEATORIA (más humano)
                    espera = random.uniform(3, 7)  # Entre 3 y 7 segundos
                    logging.warning(f" {error_msg}, reintento {intento}/{max_reintentos} en {espera:.1f}s...")
                    time.sleep(espera)
                else:
                    return {'exito': False, 'tamaño_mb': 0, 'error': error_msg}
        
        except requests.exceptions.Timeout:
            error_msg = 'Timeout (120s)'
            
            if intento < max_reintentos:
                espera = random.uniform(5, 10)
                logging.warning(f" {error_msg}, reintento {intento}/{max_reintentos} en {espera:.1f}s...")
                time.sleep(espera)
            else:
                return {'exito': False, 'tamaño_mb': 0, 'error': error_msg}
        
        except Exception as e:
            error_msg = f'{type(e).__name__}: {str(e)[:40]}'
            
            if intento < max_reintentos:
                espera = random.uniform(3, 7)
                logging.warning(f" {error_msg}, reintento {intento}/{max_reintentos} en {espera:.1f}s...")
                time.sleep(espera)
            else:
                return {'exito': False, 'tamaño_mb': 0, 'error': error_msg}
    
    return {'exito': False, 'tamaño_mb': 0, 'error': 'Error desconocido'}


# =========================================================================
# SCRAPER PRINCIPAL
# =========================================================================

def scraper_ultra_robusto(año=2025, mes=4, max_descargas=30):
    """
    Scraper con pausas más largas y comportamiento más humano
    """
    
    logging.info(f"\n{'='*70}")
    logging.info(f" SCRAPER ULTRA-ROBUSTO - {mes}/{año}")
    logging.info(f"{'='*70}\n")
    
    carpeta_final = Path("boletinesmeses")
    carpeta_final.mkdir(exist_ok=True)
    
    # Chrome
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    # User agent más reciente
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    driver = None
    
    try:
        driver = webdriver.Chrome(options=options)
        
        url = "https://diariooficial.elperuano.pe/BoletinOficial"
        logging.info(f" Navegando a: {url}")
        driver.get(url)
        time.sleep(5)
        
        # Seleccionar año
        logging.info(f" Seleccionando año: {año}...")
        select_año_element = driver.find_element(By.ID, "ddwANO")
        select_año = Select(select_año_element)
        select_año.select_by_value(str(año))
        driver.execute_script("""
            arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
        """, select_año_element)
        time.sleep(2)
        
        # Seleccionar mes
        mes_value = f"{mes:02d}"
        logging.info(f" Seleccionando mes: {mes_value}...")
        select_mes_element = driver.find_element(By.ID, "ddwMES")
        select_mes = Select(select_mes_element)
        select_mes.select_by_value(mes_value)
        driver.execute_script("""
            arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
        """, select_mes_element)
        time.sleep(2)
        
        # Buscar botón
        try:
            boton = driver.find_element(By.XPATH, "//button[contains(text(), 'Buscar')]")
            driver.execute_script("arguments[0].click();", boton)
            logging.info(f" Click en Buscar")
        except:
            logging.warning(f" No se encontró botón Buscar")
        
        time.sleep(10)
        
        # Buscar botones
        botones = driver.find_elements(By.XPATH, "//a[contains(text(),'Descarga')]")
        logging.info(f" Encontrados {len(botones)} botones\n")
        
        if len(botones) == 0:
            driver.quit()
            return {'exitosos': 0, 'fallidos': 0, 'total': 0, 'errores': ['No se encontraron botones']}
        
        # Verificar fecha
        primer_boton = botones[0]
        onclick = primer_boton.get_attribute('onclick')
        if onclick:
            fecha_match = re.search(r"(\d{1,2})/(\d{1,2})/(\d{4})", onclick)
            if fecha_match:
                dia, mes_encontrado, año_encontrado = fecha_match.groups()
                logging.info(f" Primer boletín: {dia}/{mes_encontrado}/{año_encontrado}")
                
                if int(año_encontrado) != año or int(mes_encontrado) != mes:
                    logging.error(f" Fecha incorrecta!")
                    driver.quit()
                    return {'exitosos': 0, 'fallidos': 0, 'total': 0, 'errores': ['Fecha incorrecta']}
                else:
                    logging.info(f" Fecha correcta\n")
        
        # Extraer enlaces
        enlaces = []
        for i, boton in enumerate(botones, 1):
            try:
                href = boton.get_attribute('href')
                if href:
                    onclick = boton.get_attribute('onclick')
                    fecha = None
                    if onclick:
                        fecha_match = re.search(r"(\d{1,2})/(\d{1,2})/(\d{4})", onclick)
                        if fecha_match:
                            dia, mes_num, año_num = fecha_match.groups()
                            fecha = f"{año_num}{mes_num.zfill(2)}{dia.zfill(2)}"
                    
                    enlaces.append({'url': href, 'fecha': fecha, 'numero': i})
            except:
                pass
        
        logging.info(f" Enlaces extraídos: {len(enlaces)}\n")
        driver.quit()
        driver = None
        
        # Descargar
        logging.info(f" Descargando {min(len(enlaces), max_descargas)} boletines")
        logging.info(f" Con pausas largas para evitar bloqueos...\n")
        
        exitosos = 0
        errores = []
        
        for enlace in tqdm(enlaces[:max_descargas], desc="Descargando"):
            url_pdf = enlace['url']
            fecha = enlace.get('fecha')
            numero = enlace['numero']
            
            if fecha:
                nombre = carpeta_final / f"boletin_{fecha}.pdf"
            else:
                nombre = carpeta_final / f"boletin_{año}{mes:02d}{numero:02d}.pdf"
            
            if nombre.exists():
                exitosos += 1
                continue
            
            resultado = descargar_pdf_mejorado(url_pdf, nombre, max_reintentos=3)
            
            if resultado['exito']:
                tqdm.write(f" [{numero}] {nombre.name} ({resultado['tamaño_mb']:.2f} MB)")
                exitosos += 1
            else:
                tqdm.write(f" [{numero}] {nombre.name}: {resultado['error']}")
                errores.append({'archivo': nombre.name, 'error': resultado['error']})
            
            # PAUSA MÁS LARGA Y ALEATORIA (más humano)
            pausa = random.uniform(3, 6)  # Entre 3 y 6 segundos
            time.sleep(pausa)
        
        return {
            'exitosos': exitosos,
            'fallidos': len(errores),
            'total': min(len(enlaces), max_descargas),
            'errores': errores
        }
    
    except Exception as e:
        logging.error(f" Error: {e}")
        return {'exitosos': 0, 'fallidos': 0, 'total': 0, 'errores': [str(e)]}
    
    finally:
        if driver:
            try:
                driver.quit()
            except:
                pass


# =========================================================================
# LOOP MULTI-MES
# =========================================================================

def scraper_multiples_meses(año=2025, mes_inicio=4, mes_fin=11, max_descargas_por_mes=30):
    """
    Ejecuta el scraper para múltiples meses consecutivos
    """
    
    meses_nombres = {
        1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
        5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
        9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
    }
    
    print("\n" + "="*70)
    print(" SCRAPER MULTI-MES - DIARIO EL PERUANO")
    print("="*70)
    print(f"\n  Año: {año}")
    print(f" Meses: {mes_inicio} ({meses_nombres[mes_inicio]}) a {mes_fin} ({meses_nombres[mes_fin]})")
    print(f" Max descargas por mes: {max_descargas_por_mes}")
    print("\n" + "="*70 + "\n")
    
    # Diccionario para guardar resultados
    resultados_totales = {
        'exitosos_total': 0,
        'fallidos_total': 0,
        'por_mes': {}
    }
    
    # Loop por cada mes
    for mes in range(mes_inicio, mes_fin + 1):
        
        print(f"\n{'='*70}")
        print(f" PROCESANDO: {meses_nombres[mes]} {año} (Mes {mes})")
        print("="*70 + "\n")
        
        try:
            # Ejecutar scraper para este mes
            resultado = scraper_ultra_robusto(
                año=año, 
                mes=mes, 
                max_descargas=max_descargas_por_mes
            )
            
            # Guardar resultados
            resultados_totales['por_mes'][mes] = {
                'nombre': meses_nombres[mes],
                'exitosos': resultado['exitosos'],
                'fallidos': resultado['fallidos'],
                'total': resultado['total'],
                'errores': resultado.get('errores', [])
            }
            
            resultados_totales['exitosos_total'] += resultado['exitosos']
            resultados_totales['fallidos_total'] += resultado['fallidos']
            
            print(f"\n{'─'*70}")
            print(f" {meses_nombres[mes]}: {resultado['exitosos']}/{resultado['total']} exitosos, {resultado['fallidos']} fallidos")
            print(f"{'─'*70}")
            
            # Pausa entre meses para no saturar el servidor
            if mes < mes_fin:
                print(f"\n Pausando 15 segundos antes del siguiente mes...")
                time.sleep(15)
                
        except Exception as e:
            print(f"\n ERROR en {meses_nombres[mes]}: {e}")
            resultados_totales['por_mes'][mes] = {
                'nombre': meses_nombres[mes],
                'exitosos': 0,
                'fallidos': 0,
                'total': 0,
                'error': str(e)
            }
    
    # RESUMEN FINAL
    print("\n\n" + "="*70)
    print(" RESUMEN FINAL - TODOS LOS MESES")
    print("="*70 + "\n")
    
    for mes, datos in resultados_totales['por_mes'].items():
        if 'error' in datos:
            print(f" {datos['nombre']:12} - ERROR: {datos['error']}")
        else:
            porcentaje = (datos['exitosos'] / datos['total'] * 100) if datos['total'] > 0 else 0
            print(f" {datos['nombre']:12} - {datos['exitosos']:2}/{datos['total']:2} exitosos ({porcentaje:.1f}%) | {datos['fallidos']:2} fallidos")
    
    print("\n" + "─"*70)
    print(f" TOTAL EXITOSOS:  {resultados_totales['exitosos_total']}")
    print(f" TOTAL FALLIDOS:  {resultados_totales['fallidos_total']}")
    print(f" TOTAL DESCARGAS: {resultados_totales['exitosos_total'] + resultados_totales['fallidos_total']}")
    
    if resultados_totales['exitosos_total'] + resultados_totales['fallidos_total'] > 0:
        tasa_exito = resultados_totales['exitosos_total'] / (resultados_totales['exitosos_total'] + resultados_totales['fallidos_total']) * 100
        print(f" TASA DE ÉXITO GLOBAL: {tasa_exito:.1f}%")
    
    print("="*70 + "\n")
    
    return resultados_totales


# =========================================================================
# EJECUTAR
# =========================================================================

if __name__ == "__main__":
    # CONFIGURACIÓN
    AÑO = 2025
    MES_INICIO = 4   # Abril
    MES_FIN = 11     # Noviembre
    MAX_DESCARGAS_POR_MES = 31
    
    # EJECUTAR SCRAPER MULTI-MES
    resultado_final = scraper_multiples_meses(
        año=AÑO,
        mes_inicio=MES_INICIO,
        mes_fin=MES_FIN,
        max_descargas_por_mes=MAX_DESCARGAS_POR_MES
    )
    
    print(f"\n Proceso completado!")
    print(f" Archivos guardados en: boletinesmeses/")


 SCRAPER ULTRA-ROBUSTO - 4/2025




 SCRAPER MULTI-MES - DIARIO EL PERUANO

  Año: 2025
 Meses: 4 (Abril) a 11 (Noviembre)
 Max descargas por mes: 31



 PROCESANDO: Abril 2025 (Mes 4)



 Navegando a: https://diariooficial.elperuano.pe/BoletinOficial
 Seleccionando año: 2025...
 Seleccionando mes: 04...
 Click en Buscar
 Encontrados 30 botones

 Primer boletín: 30/04/2025
 Fecha correcta

 Enlaces extraídos: 30

 Descargando 30 boletines
 Con pausas largas para evitar bloqueos...

 Descargado: boletin_20250430.pdf (20.23 MB) ?it/s]
Descargando:   0%|          | 0/30 [00:01<?, ?it/s]

 [1] boletin_20250430.pdf (20.23 MB)


 Descargado: boletin_20250429.pdf (8.13 MB)2:21,  4.89s/it]
Descargando:   3%|▎         | 1/30 [00:06<02:21,  4.89s/it]

...

 [29] boletin_20251102.pdf (7.02 MB)


 Descargado: boletin_20251101.pdf (7.17 MB)00:13, 13.61s/it]
Descargando:  97%|█████████▋| 29/30 [06:25<00:13, 13.61s/it]

 [30] boletin_20251101.pdf (7.17 MB)


Descargando: 100%|██████████| 30/30 [06:28<00:00, 12.94s/it]


──────────────────────────────────────────────────────────────────────
 Noviembre: 30/30 exitosos, 0 fallidos
──────────────────────────────────────────────────────────────────────


 RESUMEN FINAL - TODOS LOS MESES

 Abril        - 30/30 exitosos (100.0%) |  0 fallidos
 Mayo         - 31/31 exitosos (100.0%) |  0 fallidos
 Junio        - 30/30 exitosos (100.0%) |  0 fallidos
 Julio        - 31/31 exitosos (100.0%) |  0 fallidos
 Agosto       - 30/31 exitosos (96.8%) |  1 fallidos
 Septiembre   - 30/30 exitosos (100.0%) |  0 fallidos
 Octubre      - 31/31 exitosos (100.0%) |  0 fallidos
 Noviembre    - 30/30 exitosos (100.0%) |  0 fallidos

──────────────────────────────────────────────────────────────────────
 TOTAL EXITOSOS:  243
 TOTAL FALLIDOS:  1
 TOTAL DESCARGAS: 244
 TASA DE ÉXITO GLOBAL: 99.6%


 Proceso completado!
 Archivos guardados en: boletinesmeses/


## 📄 PASO 5: Extraer Texto de los PDFs

In [5]:
# =========================================================================
# EXTRACCIÓN DE TEXTO DE PDFs - PIPELINE COMPLETO
# =========================================================================

import pdfplumber
from pathlib import Path
import json
import re
from datetime import datetime
from tqdm import tqdm
import pandas as pd

# =========================================================================
# CONFIGURACIÓN
# =========================================================================

# Carpetas
CARPETA_PDFS = Path("boletinesmeses")  # ← Ajusta según tu carpeta
CARPETA_TEXTOS = Path("textos")
CARPETA_DATOS = Path("datos")

# Crear carpetasc
CARPETA_TEXTOS.mkdir(exist_ok=True)
CARPETA_DATOS.mkdir(exist_ok=True)

print(f"\n{'='*70}")
print(f" EXTRACCIÓN DE TEXTO DE PDFs")
print(f"{'='*70}\n")
print(f" Carpeta PDFs: {CARPETA_PDFS}")
print(f" Carpeta textos: {CARPETA_TEXTOS}")
print(f" Carpeta datos: {CARPETA_DATOS}\n")


# =========================================================================
# FUNCIÓN: EXTRAER TEXTO DE UN PDF
# =========================================================================

def extraer_texto_pdf(ruta_pdf):
    """
    Extrae texto de un PDF usando pdfplumber
    
    Returns:
        dict: {
            'texto': str,
            'paginas': int,
            'caracteres': int,
            'palabras': int,
            'error': str o None
        }
    """
    
    try:
        texto_completo = []
        
        with pdfplumber.open(ruta_pdf) as pdf:
            num_paginas = len(pdf.pages)
            
            for pagina in pdf.pages:
                texto_pagina = pagina.extract_text()
                
                if texto_pagina:
                    texto_completo.append(texto_pagina)
        
        # Unir todo el texto
        texto = "\n\n".join(texto_completo)
        
        # Estadísticas básicas
        num_caracteres = len(texto)
        num_palabras = len(texto.split())
        
        return {
            'texto': texto,
            'paginas': num_paginas,
            'caracteres': num_caracteres,
            'palabras': num_palabras,
            'error': None
        }
    
    except Exception as e:
        return {
            'texto': '',
            'paginas': 0,
            'caracteres': 0,
            'palabras': 0,
            'error': str(e)
        }


# =========================================================================
# FUNCIÓN: LIMPIAR TEXTO (BÁSICO)
# =========================================================================

def limpiar_texto(texto):
    """
    Limpieza básica de texto
    - Normalizar espacios
    - Quitar líneas vacías excesivas
    - Mantener estructura (importante para NER)
    """
    
    # Normalizar espacios
    texto = re.sub(r'[ \t]+', ' ', texto)
    
    # Normalizar saltos de línea (máximo 2 consecutivos)
    texto = re.sub(r'\n{3,}', '\n\n', texto)
    
    # Quitar espacios al inicio/fin de cada línea
    lineas = [linea.strip() for linea in texto.split('\n')]
    texto = '\n'.join(lineas)
    
    return texto.strip()


# =========================================================================
# FUNCIÓN: EXTRAER METADATA DEL NOMBRE
# =========================================================================

def extraer_metadata_nombre(nombre_archivo):
    """
    Extrae fecha del nombre: boletin_20250831.pdf → 2025-08-31
    """
    
    # Buscar patrón: YYYYMMDD
    match = re.search(r'(\d{4})(\d{2})(\d{2})', nombre_archivo)
    
    if match:
        año, mes, dia = match.groups()
        return {
            'fecha': f"{año}-{mes}-{dia}",
            'año': int(año),
            'mes': int(mes),
            'dia': int(dia)
        }
    
    return {
        'fecha': None,
        'año': None,
        'mes': None,
        'dia': None
    }


# =========================================================================
# PROCESO PRINCIPAL
# =========================================================================

print(" Buscando archivos PDF...")
pdfs = list(CARPETA_PDFS.glob("*.pdf"))
print(f" Encontrados: {len(pdfs)} PDFs\n")

if len(pdfs) == 0:
    print(" No se encontraron PDFs en la carpeta")
    print(f"   Verifica que la carpeta sea: {CARPETA_PDFS}\n")
    exit()

# Estructura para guardar todo
corpus = []
estadisticas = {
    'total_pdfs': len(pdfs),
    'exitosos': 0,
    'fallidos': 0,
    'total_paginas': 0,
    'total_palabras': 0,
    'total_caracteres': 0,
    'archivos': []
}

print(f"{'='*70}")
print(f" EXTRAYENDO TEXTO DE {len(pdfs)} PDFs")
print(f"{'='*70}\n")

# Procesar cada PDF
for pdf_path in tqdm(pdfs, desc="Procesando PDFs"):
    
    # Extraer texto
    resultado = extraer_texto_pdf(pdf_path)
    
    if resultado['error']:
        tqdm.write(f" {pdf_path.name}: {resultado['error']}")
        estadisticas['fallidos'] += 1
        continue
    
    # Limpiar texto
    texto_limpio = limpiar_texto(resultado['texto'])
    
    # Metadata
    metadata = extraer_metadata_nombre(pdf_path.name)
    
    # Guardar texto individual
    nombre_txt = pdf_path.stem + ".txt"
    ruta_txt = CARPETA_TEXTOS / nombre_txt
    
    with open(ruta_txt, 'w', encoding='utf-8') as f:
        f.write(texto_limpio)
    
    # Agregar al corpus
    corpus.append({
        'archivo': pdf_path.name,
        'fecha': metadata['fecha'],
        'año': metadata['año'],
        'mes': metadata['mes'],
        'dia': metadata['dia'],
        'paginas': resultado['paginas'],
        'palabras': resultado['palabras'],
        'caracteres': resultado['caracteres'],
        'texto': texto_limpio,
        'ruta_txt': str(ruta_txt)
    })
    
    # Actualizar estadísticas
    estadisticas['exitosos'] += 1
    estadisticas['total_paginas'] += resultado['paginas']
    estadisticas['total_palabras'] += resultado['palabras']
    estadisticas['total_caracteres'] += resultado['caracteres']
    
    estadisticas['archivos'].append({
        'archivo': pdf_path.name,
        'fecha': metadata['fecha'],
        'paginas': resultado['paginas'],
        'palabras': resultado['palabras']
    })
    
    tqdm.write(f" {pdf_path.name}: {resultado['paginas']} págs, {resultado['palabras']:,} palabras")

print(f"\n{'='*70}")
print(f" GUARDANDO DATOS")
print(f"{'='*70}\n")

# =========================================================================
# GUARDAR: JSON con todo (para NER posterior)
# =========================================================================

print(" Guardando JSON completo...", end=" ")
ruta_json = CARPETA_DATOS / "todos_textos.json"

with open(ruta_json, 'w', encoding='utf-8') as f:
    json.dump(corpus, f, ensure_ascii=False, indent=2)

print(f" {ruta_json}")

# =========================================================================
# GUARDAR: Corpus completo (todo el texto junto)
# =========================================================================

print(" Guardando corpus completo...", end=" ")
ruta_corpus = CARPETA_DATOS / "corpus_completo.txt"

with open(ruta_corpus, 'w', encoding='utf-8') as f:
    for doc in corpus:
        f.write(f"\n{'='*70}\n")
        f.write(f"ARCHIVO: {doc['archivo']}\n")
        f.write(f"FECHA: {doc['fecha']}\n")
        f.write(f"{'='*70}\n\n")
        f.write(doc['texto'])
        f.write("\n\n")

print(f" {ruta_corpus}")

# =========================================================================
# GUARDAR: Estadísticas
# =========================================================================

print("📄 Guardando estadísticas...", end=" ")

# Agregar promedios
estadisticas['promedio_paginas'] = estadisticas['total_paginas'] / estadisticas['exitosos'] if estadisticas['exitosos'] > 0 else 0
estadisticas['promedio_palabras'] = estadisticas['total_palabras'] / estadisticas['exitosos'] if estadisticas['exitosos'] > 0 else 0

ruta_stats = CARPETA_DATOS / "estadisticas.json"

with open(ruta_stats, 'w', encoding='utf-8') as f:
    json.dump(estadisticas, f, ensure_ascii=False, indent=2)

print(f" {ruta_stats}")

# =========================================================================
# GUARDAR: CSV con metadata
# =========================================================================

print(" Guardando CSV con metadata...", end=" ")
ruta_csv = CARPETA_DATOS / "metadata_boletines.csv"

df = pd.DataFrame([{
    'archivo': doc['archivo'],
    'fecha': doc['fecha'],
    'año': doc['año'],
    'mes': doc['mes'],
    'dia': doc['dia'],
    'paginas': doc['paginas'],
    'palabras': doc['palabras'],
    'caracteres': doc['caracteres']
} for doc in corpus])

df.to_csv(ruta_csv, index=False, encoding='utf-8')

print(f" {ruta_csv}")

# =========================================================================
# RESUMEN FINAL
# =========================================================================

print(f"\n{'='*70}")
print(f" RESUMEN FINAL")
print(f"{'='*70}\n")

print(f" PDFs procesados: {estadisticas['exitosos']}/{estadisticas['total_pdfs']}")
print(f" PDFs fallidos: {estadisticas['fallidos']}")
print(f"\n Estadísticas del corpus:")
print(f"   Total páginas: {estadisticas['total_paginas']:,}")
print(f"   Total palabras: {estadisticas['total_palabras']:,}")
print(f"   Total caracteres: {estadisticas['total_caracteres']:,}")
print(f"\n Promedios:")
print(f"   Páginas/PDF: {estadisticas['promedio_paginas']:.1f}")
print(f"   Palabras/PDF: {estadisticas['promedio_palabras']:,.0f}")

print(f"\n{'='*70}")
print(f" ARCHIVOS GENERADOS")
print(f"{'='*70}\n")

print(f" {CARPETA_TEXTOS}/")
print(f"   ├── {estadisticas['exitosos']} archivos .txt (uno por PDF)")
print(f"\n {CARPETA_DATOS}/")
print(f"   ├── todos_textos.json (corpus completo + metadata)")
print(f"   ├── corpus_completo.txt (todo el texto unido)")
print(f"   ├── estadisticas.json (stats del corpus)")
print(f"   └── metadata_boletines.csv (tabla de metadatos)")

print(f"\n{'='*70}")
print(f" EXTRACCIÓN COMPLETADA")
print(f"{'='*70}\n")

print(f" Próximo paso: NER (Named Entity Recognition)")
print(f"   Archivos listos en: {CARPETA_DATOS}/todos_textos.json\n")


 EXTRACCIÓN DE TEXTO DE PDFs

 Carpeta PDFs: boletinesmeses
 Carpeta textos: textos
 Carpeta datos: datos

 Buscando archivos PDF...
 Encontrados: 243 PDFs

 EXTRAYENDO TEXTO DE 243 PDFs



Procesando PDFs:   0%|          | 1/243 [00:37<2:30:55, 37.42s/it]

 boletin_20250401.pdf: 20 págs, 75,955 palabras


Procesando PDFs:   1%|          | 2/243 [01:37<3:22:50, 50.50s/it]

 boletin_20250402.pdf: 44 págs, 131,555 palabras


...

 boletin_20251128.pdf: 48 págs, 149,939 palabras


Procesando PDFs: 100%|█████████▉| 242/243 [4:25:16<01:03, 63.02s/it]

 boletin_20251129.pdf: 16 págs, 32,935 palabras


Procesando PDFs: 100%|██████████| 243/243 [4:26:08<00:00, 65.71s/it]


 boletin_20251130.pdf: 13 págs, 17,013 palabras

 GUARDANDO DATOS

 datos\todos_textos.json... 
 datos\corpus_completo.txt... 
📄 Guardando estadísticas...  datos\estadisticas.json
 Guardando CSV con metadata...  datos\metadata_boletines.csv

 RESUMEN FINAL

 PDFs procesados: 243/243
 PDFs fallidos: 0

 Estadísticas del corpus:
   Total páginas: 7,839
   Total palabras: 22,199,814
   Total caracteres: 117,463,952

 Promedios:
   Páginas/PDF: 32.3
   Palabras/PDF: 91,357

 ARCHIVOS GENERADOS

 textos/
   ├── 243 archivos .txt (uno por PDF)

 datos/
   ├── todos_textos.json (corpus completo + metadata)
   ├── corpus_completo.txt (todo el texto unido)
   ├── estadisticas.json (stats del corpus)
   └── metadata_boletines.csv (tabla de metadatos)

 EXTRACCIÓN COMPLETADA

 Próximo paso: NER (Named Entity Recognition)
   Archivos listos en: datos/todos_textos.json



## LIMPIEZA DE CORPUS Y PREPARACION PARA NER

In [ ]:

# =========================================================================
# LIMPIEZA DE CORPUS Y PREPARACION PARA NER
# =========================================================================
# Lee datos/todos_textos.json y crea:
#   - datos_limpios/corpus_limpio.json
#   - datos_limpios/estadisticas_limpieza.json
#   - datos_limpios/textos_para_ner.txt
# =========================================================================

import json
from pathlib import Path
from bs4 import BeautifulSoup
import re

print("\n" + "="*70)
print(" LIMPIEZA DE CORPUS")
print("="*70 + "\n")

# =========================================================================
# CONFIGURACION
# =========================================================================

CARPETA_DATOS = Path("datos")
CARPETA_LIMPIO = Path("datos_limpios")
CARPETA_LIMPIO.mkdir(exist_ok=True)

# =========================================================================
# FUNCION: LIMPIAR TEXTO
# =========================================================================

def limpiar_texto(texto):
    """
    Limpia HTML y caracteres especiales del texto
    """
    
    if not texto or len(texto.strip()) == 0:
        return ""
    
    # 1. Eliminar HTML con BeautifulSoup
    try:
        soup = BeautifulSoup(texto, 'html.parser')
        texto = soup.get_text(separator=' ')
    except:
        # Si falla BeautifulSoup, usar regex simple
        texto = re.sub(r'<[^>]+>', '', texto)
    
    # 2. Normalizar espacios en blanco
    texto = re.sub(r'\s+', ' ', texto)
    
    # 3. Eliminar caracteres de control (excepto saltos de linea)
    texto = re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]', '', texto)
    
    # 4. Normalizar saltos de linea
    texto = re.sub(r'\n\s*\n\s*\n+', '\n\n', texto)
    
    # 5. Limpiar al inicio y final
    texto = texto.strip()
    
    return texto

# =========================================================================
# CARGAR CORPUS ORIGINAL
# =========================================================================

print(" Cargando corpus original...")
ruta_original = CARPETA_DATOS / "todos_textos.json"

if not ruta_original.exists():
    print(f" ERROR: No se encontro {ruta_original}")
    print("   Ejecuta primero la celda de extraccion de PDFs")
else:
    with open(ruta_original, 'r', encoding='utf-8') as f:
        corpus_original = json.load(f)
    
    print(f" Cargados {len(corpus_original)} documentos\n")
    
    # =====================================================================
    # LIMPIAR CADA DOCUMENTO
    # =====================================================================
    
    print(" Limpiando textos...")
    
    corpus_limpio = []
    textos_concatenados = []
    
    total_caracteres_originales = 0
    total_caracteres_limpios = 0
    total_palabras = 0
    
    for doc in corpus_original:
        # Limpiar texto
        texto_limpio = limpiar_texto(doc['texto'])
        
        # Crear documento limpio
        doc_limpio = {
            'archivo': doc['archivo'],
            'fecha': doc['fecha'],
            'año': doc['año'],
            'mes': doc['mes'],
            'texto_limpio': texto_limpio,
            'palabras': len(texto_limpio.split()),
            'caracteres': len(texto_limpio)
        }
        
        corpus_limpio.append(doc_limpio)
        textos_concatenados.append(texto_limpio)
        
        # Estadisticas
        total_caracteres_originales += doc['caracteres']
        total_caracteres_limpios += len(texto_limpio)
        total_palabras += len(texto_limpio.split())
    
    print(f" {len(corpus_limpio)} documentos limpiados\n")
    
    # =====================================================================
    # GUARDAR ARCHIVOS
    # =====================================================================
    
    print("="*70)
    print(" GUARDANDO ARCHIVOS")
    print("="*70 + "\n")
    
    # 1. corpus_limpio.json
    print(" 1. corpus_limpio.json...", end=" ")
    ruta_limpio = CARPETA_LIMPIO / "corpus_limpio.json"
    
    with open(ruta_limpio, 'w', encoding='utf-8') as f:
        json.dump(corpus_limpio, f, ensure_ascii=False, indent=2)
    
    tamano_mb = ruta_limpio.stat().st_size / (1024*1024)
    print(f"OK ({tamano_mb:.1f} MB)")
    
    # 2. textos_para_ner.txt
    print(" 2. textos_para_ner.txt...", end=" ")
    ruta_ner = CARPETA_LIMPIO / "textos_para_ner.txt"
    
    with open(ruta_ner, 'w', encoding='utf-8') as f:
        f.write("\n\n".join(textos_concatenados))
    
    tamano_kb = ruta_ner.stat().st_size / 1024
    print(f"OK ({tamano_kb:.1f} KB)")
    
    # 3. estadisticas_limpieza.json
    print(" 3. estadisticas_limpieza.json...", end=" ")
    
    estadisticas = {
        'total_documentos': len(corpus_limpio),
        'total_palabras': total_palabras,
        'total_caracteres_originales': total_caracteres_originales,
        'total_caracteres_limpios': total_caracteres_limpios,
        'reduccion_caracteres': total_caracteres_originales - total_caracteres_limpios,
        'porcentaje_reduccion': round((total_caracteres_originales - total_caracteres_limpios) / total_caracteres_originales * 100, 2),
        'promedio_palabras_por_doc': total_palabras // len(corpus_limpio),
        'promedio_caracteres_por_doc': total_caracteres_limpios // len(corpus_limpio)
    }
    
    ruta_stats = CARPETA_LIMPIO / "estadisticas_limpieza.json"
    
    with open(ruta_stats, 'w', encoding='utf-8') as f:
        json.dump(estadisticas, f, ensure_ascii=False, indent=2)
    
    print("OK")
    
    # =====================================================================
    # ESTADISTICAS
    # =====================================================================
    
    print(f"\n{'='*70}")
    print(" ESTADISTICAS DE LIMPIEZA")
    print("="*70 + "\n")
    
    print(f" Documentos procesados: {len(corpus_limpio)}")
    print(f" Total palabras: {total_palabras:,}")
    print(f" Caracteres originales: {total_caracteres_originales:,}")
    print(f" Caracteres limpios: {total_caracteres_limpios:,}")
    print(f" Reduccion: {estadisticas['reduccion_caracteres']:,} caracteres ({estadisticas['porcentaje_reduccion']}%)")
    print(f" Promedio palabras/doc: {estadisticas['promedio_palabras_por_doc']:,}")
    
    # Mostrar muestra
    print(f"\n{'='*70}")
    print(" MUESTRA DE TEXTO LIMPIO")
    print("="*70 + "\n")
    
    muestra = corpus_limpio[0]['texto_limpio'][:250]
    print(f"Documento: {corpus_limpio[0]['archivo']}")
    print(f"Fecha: {corpus_limpio[0]['fecha']}\n")
    print(muestra)
    print("...\n")
    
    print("="*70)
    print(" LIMPIEZA COMPLETADA")
    print("="*70 + "\n")
    
    print(" Archivos generados en datos_limpios/:")
    print("   - corpus_limpio.json")
    print("   - textos_para_ner.txt")
    print("   - estadisticas_limpieza.json")
    print("\n PROXIMO PASO:")
    print("   Ejecutar celda de NER (Named Entity Recognition)")

print("\n" + "="*70 + "\n")

## PASO 6: Extracción de Entidades (NER)

Usaremos spaCy con el modelo en español para detectar entidades.

In [3]:
# =========================================================================
# NER - CON FILTROS Y REGLAS PERSONALIZADAS
# =========================================================================

import spacy
from spacy.matcher import Matcher
from spacy.pipeline import EntityRuler
import json
from pathlib import Path
from tqdm import tqdm
from collections import Counter, defaultdict
import pandas as pd
import re

print("\n" + "="*70)
print(" NER - CON REGLAS PERSONALIZADAS")
print("="*70 + "\n")

# =========================================================================
# CONFIGURACIÓN
# =========================================================================

CARPETA_DATOS = Path("datos_limpios")
CARPETA_NER = Path("entidades_mejorado")
CARPETA_NER.mkdir(exist_ok=True)

# Cargar modelo
print(" Cargando modelo spaCy...")
nlp = spacy.load("es_core_news_md")

# =========================================================================
# LISTAS PERSONALIZADAS - DOMINIO LEGAL PERUANO
# =========================================================================

# Organizaciones conocidas (gubernamentales, etc.)
ORGANIZACIONES_CONOCIDAS = {
    'sunat', 'mef', 'pcm', 'minsa', 'minedu', 'mtc', 'mindef', 'minjus',
    'ministerio de economía', 'ministerio de salud', 'ministerio de educación',
    'congreso de la república', 'tribunal constitucional', 'poder judicial',
    'contraloría general', 'defensoría del pueblo', 'ministerio público',
    'banco central de reserva', 'bcr', 'sbs', 'superintendencia',
    'essalud', 'reniec', 'onpe', 'jne', 'indecopi', 'osinergmin',
    'sunass', 'osiptel', 'sunafil', 'senace', 'sernanp', 'senasa',
    'inia', 'ana', 'serfor', 'dicapi', 'produce', 'midagri', 'minam'
}

# Palabras que NO son entidades (blacklist)
PALABRAS_BLACKLIST = {
    # Verbos comunes en documentos legales
    'siendo', 'considerando', 'teniendo', 'visto', 'estando', 'dado',
    'solicita', 'solicitando', 'dispone', 'aprueba', 'modifica',
    # Preposiciones y conjunciones
    'con', 'sin', 'por', 'para', 'sobre', 'mediante', 'según',
    # Términos legales comunes
    'propiedad', 'dominio', 'calle', 'avenida', 'jirón', 'ubicado',
    'notarial', 'sucesión intestada', 'fallecido', 'estado civil',
    # Referencias
    'art', 'artículo', 'inciso', 'literal', 'numeral', 'decreto',
    'la ley', 'el código', 'el reglamento', 'se ha', 'ha sido',
    # Otros
    'distrito', 'provincia', 'región', 'departamento'  # Estos pueden ser lugares pero no como están
}

# Patrones para identificar (números de resolución, decretos, etc.)
PATRONES_LEGALES = [
    # Decretos Supremos
    (r'DECRETO\s+SUPREMO\s+N[°º]?\s*(\d{3,4}-\d{4}-\w+)', 'DECRETO_SUPREMO'),
    # Resoluciones
    (r'RESOLUCIÓN\s+(?:MINISTERIAL|DIRECTORAL|JEFATURAL)\s+N[°º]?\s*(\d{3,4}-\d{4}-\w+)', 'RESOLUCION'),
    # Leyes
    (r'LEY\s+N[°º]?\s*(\d{4,5})', 'LEY'),
    # RUC
    (r'RUC\s+N[°º]?\s*(\d{11})', 'RUC'),
    # Montos en soles
    (r'S/\.?\s*([\d,]+\.?\d*)', 'MONTO_SOLES'),
    # Montos en dólares
    (r'US\$\s*([\d,]+\.?\d*)', 'MONTO_DOLARES'),
]

# =========================================================================
# AGREGAR ENTITY RULER CON PATRONES
# =========================================================================

print(" Configurando reglas personalizadas...")

# Crear Entity Ruler
ruler = nlp.add_pipe("entity_ruler", before="ner")

# Patrones para organizaciones conocidas
patterns = []

for org in ORGANIZACIONES_CONOCIDAS:
    patterns.append({"label": "ORG", "pattern": org})
    patterns.append({"label": "ORG", "pattern": org.upper()})
    patterns.append({"label": "ORG", "pattern": org.title()})

# Agregar patrones de decretos y resoluciones
patterns.extend([
    {"label": "DECRETO", "pattern": [
        {"TEXT": {"REGEX": "DECRETO"}},
        {"TEXT": {"REGEX": "SUPREMO|LEGISLATIVO|DE URGENCIA"}},
        {"TEXT": {"REGEX": "N[°º]?"}},
        {"TEXT": {"REGEX": r"\d{3,4}-\d{4}-\w+"}}
    ]},
    {"label": "RESOLUCION", "pattern": [
        {"TEXT": {"REGEX": "RESOLUCIÓN"}},
        {"TEXT": {"REGEX": "MINISTERIAL|DIRECTORAL|JEFATURAL"}},
        {"TEXT": {"REGEX": "N[°º]?"}},
        {"TEXT": {"REGEX": r"\d{3,4}-\d{4}-\w+"}}
    ]},
    {"label": "LEY", "pattern": [
        {"TEXT": "Ley"},
        {"TEXT": {"REGEX": "N[°º]?"}},
        {"TEXT": {"REGEX": r"\d{4,5}"}}
    ]},
])

ruler.add_patterns(patterns)

print(f" {len(patterns)} patrones agregados\n")

# =========================================================================
# FUNCIÓN: FILTRAR ENTIDADES INVÁLIDAS
# =========================================================================

def filtrar_entidades(entidades):
    """
    Filtra entidades usando blacklist y reglas de validación
    """
    
    entidades_validas = []
    
    for ent in entidades:
        texto = ent['texto'].strip()
        texto_lower = texto.lower()
        tipo = ent['tipo']
        
        # FILTRO 1: Blacklist
        if texto_lower in PALABRAS_BLACKLIST:
            continue
        
        # FILTRO 2: Muy corto (menos de 3 caracteres)
        if len(texto) < 3:
            continue
        
        # FILTRO 3: Solo números (excepto si es tipo específico)
        if texto.replace(',', '').replace('.', '').isdigit() and tipo not in ['CARDINAL', 'MONEY']:
            continue
        
        # FILTRO 4: Palabras comunes que no son organizaciones
        if tipo == 'ORG':
            # No es organización si es una palabra común
            palabras_comunes_org = {'calle', 'avenida', 'jirón', 'pasaje', 'ubicado', 'propiedad'}
            if texto_lower in palabras_comunes_org:
                continue
            
            # Debe tener al menos 4 caracteres para ser organización válida
            if len(texto) < 4:
                continue
        
        # FILTRO 5: Lugares inválidos
        if tipo == 'LOC':
            lugares_invalidos = {'la ley', 'se ha', 'art', 'fallecido', 'estado civil', 'el código'}
            if texto_lower in lugares_invalidos:
                continue
        
        # FILTRO 6: Personas inválidas
        if tipo == 'PER':
            personas_invalidas = {'distrito', 'sucesión intestada', 'dominio', 'notario público', 'notaría'}
            if texto_lower in personas_invalidas:
                continue
            
            # Probablemente no es persona si está todo en mayúsculas y tiene más de una palabra
            if texto.isupper() and len(texto.split()) > 2:
                continue
        
        # Si pasa todos los filtros, agregar
        entidades_validas.append(ent)
    
    return entidades_validas


# =========================================================================
# FUNCIÓN: NORMALIZAR ORGANIZACIONES
# =========================================================================

def normalizar_organizacion(texto):
    """
    Normaliza nombres de organizaciones a su forma estándar
    """
    
    texto_lower = texto.lower()
    
    normalizaciones = {
        'sunat': 'SUNAT',
        'superintendencia nacional de aduanas': 'SUNAT',
        'mef': 'Ministerio de Economía y Finanzas',
        'ministerio de economía': 'Ministerio de Economía y Finanzas',
        'pcm': 'Presidencia del Consejo de Ministros',
        'minsa': 'Ministerio de Salud',
        'ministerio de salud': 'Ministerio de Salud',
        'minedu': 'Ministerio de Educación',
        'ministerio de educación': 'Ministerio de Educación',
        'congreso': 'Congreso de la República',
        'tribunal constitucional': 'Tribunal Constitucional',
        'tc': 'Tribunal Constitucional',
        'poder judicial': 'Poder Judicial',
        'pj': 'Poder Judicial',
        'contraloría': 'Contraloría General de la República',
        'essalud': 'EsSalud',
        'sbs': 'Superintendencia de Banca, Seguros y AFP',
        'bcr': 'Banco Central de Reserva del Perú',
        'banco central': 'Banco Central de Reserva del Perú',
        'reniec': 'RENIEC',
        'onpe': 'ONPE',
        'jne': 'Jurado Nacional de Elecciones',
        'indecopi': 'INDECOPI',
    }
    
    # Buscar normalización
    for clave, valor in normalizaciones.items():
        if clave in texto_lower:
            return valor
    
    return texto


# =========================================================================
# FUNCIÓN: EXTRAER ENTIDADES CON TODO LO MEJORADO
# =========================================================================

def extraer_entidades_mejorado(texto, nlp):
    """
    Extrae entidades con modelo mejorado + filtros
    """
    
    # Limitar longitud
    max_length = 1000000
    if len(texto) > max_length:
        texto = texto[:max_length]
    
    # Procesar con spaCy (ahora con entity ruler)
    doc = nlp(texto)
    
    # Extraer entidades
    entidades = []
    
    for ent in doc.ents:
        entidades.append({
            'texto': ent.text,
            'tipo': ent.label_,
            'inicio': ent.start_char,
            'fin': ent.end_char
        })
    
    # Filtrar entidades inválidas
    entidades_filtradas = filtrar_entidades(entidades)
    
    # Normalizar organizaciones
    for ent in entidades_filtradas:
        if ent['tipo'] == 'ORG':
            ent['texto_normalizado'] = normalizar_organizacion(ent['texto'])
        else:
            ent['texto_normalizado'] = ent['texto']
    
    # Extraer entidades con regex (decretos, leyes, montos)
    entidades_regex = extraer_con_regex(texto)
    entidades_filtradas.extend(entidades_regex)
    
    return entidades_filtradas


# =========================================================================
# FUNCIÓN: EXTRAER CON REGEX (decretos, leyes, montos)
# =========================================================================

def extraer_con_regex(texto):
    """
    Extrae entidades usando expresiones regulares
    """
    
    entidades = []
    
    for patron, tipo in PATRONES_LEGALES:
        matches = re.finditer(patron, texto, re.IGNORECASE)
        
        for match in matches:
            entidades.append({
                'texto': match.group(0),
                'texto_normalizado': match.group(0),
                'tipo': tipo,
                'inicio': match.start(),
                'fin': match.end()
            })
    
    return entidades


# =========================================================================
# CARGAR Y PROCESAR CORPUS
# =========================================================================

print(" Cargando corpus...")
ruta_corpus = CARPETA_DATOS / "corpus_limpio.json"

with open(ruta_corpus, 'r', encoding='utf-8') as f:
    corpus = json.load(f)

print(f" Cargados {len(corpus)} documentos\n")

print("="*70)
print(" EXTRAYENDO ENTIDADES (versión mejorada)")
print("="*70 + "\n")

# Procesar
todas_entidades = []
contador_tipos = Counter()
contador_entidades = Counter()

for doc in tqdm(corpus, desc="Procesando"):
    
    # Extraer con versión mejorada
    entidades = extraer_entidades_mejorado(doc['texto_limpio'], nlp)
    
    for ent in entidades:
        todas_entidades.append({
            'archivo': doc['archivo'],
            'fecha': doc['fecha'],
            'año': doc['año'],
            'mes': doc['mes'],
            **ent
        })
        
        contador_tipos[ent['tipo']] += 1
        contador_entidades[ent['texto_normalizado']] += 1

print(f"\n Total entidades: {len(todas_entidades):,}\n")

# =========================================================================
# ANÁLISIS Y GUARDADO (igual que antes)
# =========================================================================

# Top por tipo
entidades_por_tipo = defaultdict(list)
for ent in todas_entidades:
    entidades_por_tipo[ent['tipo']].append(ent['texto_normalizado'])

print("="*70)
print(" TOP 10 ENTIDADES POR TIPO")
print("="*70 + "\n")

for tipo in ['ORG', 'PER', 'LOC', 'DECRETO', 'LEY', 'RESOLUCION']:
    if tipo in entidades_por_tipo:
        counter = Counter(entidades_por_tipo[tipo])
        top_10 = counter.most_common(10)
        
        print(f" {tipo} (Total: {len(entidades_por_tipo[tipo]):,}):")
        for i, (ent, freq) in enumerate(top_10, 1):
            print(f"   {i:2d}. {ent[:50]:50s} ({freq:,})")
        print()

# Guardar (mismo código que antes)
print("="*70)
print(" GUARDANDO RESULTADOS")
print("="*70 + "\n")

# JSON
ruta_json = CARPETA_NER / "entidades_mejorado.json"
with open(ruta_json, 'w', encoding='utf-8') as f:
    json.dump(todas_entidades, f, ensure_ascii=False, indent=2)
print(f" JSON guardado ({ruta_json.stat().st_size / (1024*1024):.1f} MB)")

# CSV
ruta_csv = CARPETA_NER / "entidades_mejorado.csv"
df = pd.DataFrame(todas_entidades)
df.to_csv(ruta_csv, index=False, encoding='utf-8')
print(f" CSV guardado\n")

print("="*70)
print(" NER COMPLETADO")
print("="*70 + "\n")


 NER - CON REGLAS PERSONALIZADAS

 Cargando modelo spaCy...
 Configurando reglas personalizadas...
 123 patrones agregados

 Cargando corpus...
 Cargados 243 documentos

 EXTRAYENDO ENTIDADES (versión mejorada)



Procesando: 100%|██████████| 243/243 [57:57<00:00, 14.31s/it] 



 Total entidades: 3,043,025

 TOP 10 ENTIDADES POR TIPO

 ORG (Total: 729,075):
    1. RENIEC                                             (9,031)
    2. SUCESION INTESTADA                                 (7,624)
    3. SUNAT                                              (7,522)
    4. FRENTE                                             (6,555)
    5. COMUNICO                                           (6,443)
    6. CONFORMIDAD                                        (5,236)
    7. LO QUE                                             (4,906)
    8. OFICINA                                            (4,801)
    9. PREDIOS                                            (4,119)
   10. CALIDAD                                            (4,027)

 PER (Total: 180,463):
    1. SUCESIÓN INTESTADA.-                               (3,504)
    2. DOMINIO.-                                          (2,191)
    3. Notario                                            (980)
    4. Jesús María                     

In [5]:
"""
# =========================================================================
# EXTRACTOR ESTRUCTURADO - DIARIO EL PERUANO
# =========================================================================
# Extrae información útil: remates, edictos, licitaciones, JGA, etc.
# =========================================================================

import re
import json
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from datetime import datetime

print("\n" + "="*70)
print(" EXTRACTOR ESTRUCTURADO - DIARIO EL PERUANO")
print("="*70 + "\n")

# =========================================================================
# CONFIGURACIÓN
# =========================================================================

CARPETA_DATOS = Path("datos_limpios")
CARPETA_EXTRAIDO = Path("informacion_extraida")
CARPETA_EXTRAIDO.mkdir(exist_ok=True)

# =========================================================================
# PATRONES DE EXTRACCIÓN
# =========================================================================

# 1. REMATES PÚBLICOS
PATRON_REMATE = {
    'titulo': r'REMATE|SUBASTA|VENTA JUDICIAL',
    'bien': r'(?:INMUEBLE|VEHICULO|MAQUINARIA|BIEN).*?(?=UBICACIÓN|UBICADO|BASE)',
    'ubicacion': r'UBICACIÓN?:?\s*([^\n]+)',
    'base': r'BASE:?\s*S/\.?\s*([\d,]+\.?\d*)',
    'fecha': r'FECHA:?\s*(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})',
    'hora': r'HORA:?\s*(\d{1,2}:\d{2})',
    'expediente': r'EXPEDIENTE:?\s*N[°º]?\s*([\d-]+)',
}

# 2. EDICTOS
PATRON_EDICTO = {
    'tipo': r'EDICTO\s+(?:JUDICIAL|NOTARIAL|MATRIMONIAL)?',
    'demandante': r'DEMANDANTE:?\s*([^\n]+)',
    'demandado': r'DEMANDADO:?\s*([^\n]+)',
    'expediente': r'EXPEDIENTE:?\s*N[°º]?\s*([\d-]+)',
    'juzgado': r'JUZGADO:?\s*([^\n]+)',
}

# 3. LICITACIONES PÚBLICAS
PATRON_LICITACION = {
    'titulo': r'LICITACIÓN PÚBLICA|CONCURSO PÚBLICO|ADJUDICACIÓN',
    'entidad': r'ENTIDAD:?\s*([^\n]+)',
    'objeto': r'OBJETO:?\s*([^\n]+)',
    'valor': r'VALOR REFERENCIAL:?\s*S/\.?\s*([\d,]+\.?\d*)',
    'fecha_bases': r'VENTA DE BASES:?\s*(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})',
}

# 4. CONVOCATORIA JUNTA GENERAL DE ACCIONISTAS
PATRON_JGA = {
    'empresa': r'(?:CONVOCA|SE CONVOCA).*?JUNTA GENERAL.*?DE\s+([A-ZÁÉÍÓÚ\s\.]+?)(?:\s+S\.?A\.?|\s+SAC)',
    'fecha_junta': r'(?:JUNTA.*?)?(\d{1,2})\s+DE\s+(\w+)\s+(?:DE|DEL)?\s*(\d{4})',
    'hora': r'HORA:?\s*(\d{1,2}:\d{2})',
    'lugar': r'LUGAR:?\s*([^\n]+)',
    'agenda': r'AGENDA:?\s*([^\n]+(?:\n[^\n]+){0,5})',
}

# 5. FUSIÓN DE EMPRESAS
PATRON_FUSION = {
    'titulo': r'FUSIÓN|AVISO.*?FUSIÓN',
    'absorbente': r'(?:EMPRESA\s+)?ABSORBENTE:?\s*([^\n]+)',
    'absorbida': r'(?:EMPRESA\s+)?ABSORBIDA:?\s*([^\n]+)',
    'fecha': r'FECHA.*?FUSIÓN:?\s*(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})',
}

# 6. DISOLUCIÓN Y LIQUIDACIÓN
PATRON_DISOLUCION = {
    'titulo': r'DISOLUCIÓN.*?LIQUIDACIÓN|LIQUIDACIÓN.*?EMPRESA',
    'empresa': r'(?:EMPRESA|RAZÓN SOCIAL):?\s*([^\n]+)',
    'ruc': r'RUC:?\s*N[°º]?\s*(\d{11})',
    'liquidador': r'LIQUIDADOR:?\s*([^\n]+)',
}

# 7. REQUERIMIENTOS SBS
PATRON_SBS = {
    'titulo': r'SUPERINTENDENCIA DE BANCA|SBS|REQUERIMIENTO.*?SBS',
    'entidad': r'(?:ENTIDAD|EMPRESA):?\s*([^\n]+)',
    'tipo': r'(?:TIPO|MATERIA):?\s*([^\n]+)',
    'expediente': r'EXPEDIENTE:?\s*N[°º]?\s*([\d-]+)',
}

# =========================================================================
# FUNCIÓN: DETECTAR SECCIONES DEL BOLETÍN
# =========================================================================

def detectar_secciones(texto):
    """
    Identifica secciones del boletín por títulos
    
    Returns:
        dict: {
            'REMATES': [texto de sección],
            'EDICTOS': [texto de sección],
            ...
        }
    """
    
    secciones = {
        'REMATES': [],
        'EDICTOS': [],
        'LICITACIONES': [],
        'CONVOCATORIAS': [],
        'FUSIONES': [],
        'DISOLUCIONES': [],
        'SBS': [],
        'OTROS': []
    }
    
    # Patrones de títulos de sección (mayúsculas, centrados, etc.)
    titulo_seccion = r'\n\s*([A-ZÁÉÍÓÚ\s]{10,})\s*\n'
    
    matches = list(re.finditer(titulo_seccion, texto))
    
    for i, match in enumerate(matches):
        titulo = match.group(1).strip()
        inicio = match.end()
        fin = matches[i+1].start() if i+1 < len(matches) else len(texto)
        contenido = texto[inicio:fin]
        
        # Clasificar sección
        if 'REMATE' in titulo or 'SUBASTA' in titulo:
            secciones['REMATES'].append(contenido)
        elif 'EDICTO' in titulo:
            secciones['EDICTOS'].append(contenido)
        elif 'LICITACIÓN' in titulo or 'CONCURSO' in titulo:
            secciones['LICITACIONES'].append(contenido)
        elif 'JUNTA' in titulo and 'ACCIONISTA' in titulo:
            secciones['CONVOCATORIAS'].append(contenido)
        elif 'FUSIÓN' in titulo:
            secciones['FUSIONES'].append(contenido)
        elif 'DISOLUCIÓN' in titulo or 'LIQUIDACIÓN' in titulo:
            secciones['DISOLUCIONES'].append(contenido)
        elif 'SUPERINTENDENCIA' in titulo or 'SBS' in titulo:
            secciones['SBS'].append(contenido)
        else:
            secciones['OTROS'].append(contenido)
    
    return secciones


# =========================================================================
# FUNCIÓN: EXTRAER REMATES
# =========================================================================

def extraer_remates(texto):
    """
    Extrae información estructurada de remates
    """
    
    remates = []
    
    # Dividir por avisos individuales (usualmente separados por líneas)
    avisos = re.split(r'\n\s*-{3,}\s*\n', texto)
    
    for aviso in avisos:
        # Verificar que sea un remate
        if not re.search(PATRON_REMATE['titulo'], aviso, re.IGNORECASE):
            continue
        
        # Extraer datos
        remate = {
            'tipo': 'REMATE',
            'bien': None,
            'ubicacion': None,
            'base': None,
            'fecha': None,
            'hora': None,
            'expediente': None,
            'texto_completo': aviso[:500]  # Primeros 500 caracteres
        }
        
        # Ubicación
        match = re.search(PATRON_REMATE['ubicacion'], aviso, re.IGNORECASE)
        if match:
            remate['ubicacion'] = match.group(1).strip()
        
        # Base
        match = re.search(PATRON_REMATE['base'], aviso, re.IGNORECASE)
        if match:
            remate['base'] = match.group(1).replace(',', '')
        
        # Fecha
        match = re.search(PATRON_REMATE['fecha'], aviso)
        if match:
            remate['fecha'] = match.group(1)
        
        # Hora
        match = re.search(PATRON_REMATE['hora'], aviso)
        if match:
            remate['hora'] = match.group(1)
        
        # Expediente
        match = re.search(PATRON_REMATE['expediente'], aviso, re.IGNORECASE)
        if match:
            remate['expediente'] = match.group(1)
        
        # Solo agregar si tiene al menos ubicación o base
        if remate['ubicacion'] or remate['base']:
            remates.append(remate)
    
    return remates


# =========================================================================
# FUNCIÓN: EXTRAER CONVOCATORIAS JGA
# =========================================================================

def extraer_jga(texto):
    """
    Extrae convocatorias de Juntas Generales de Accionistas
    """
    
    convocatorias = []
    
    # Dividir por avisos
    avisos = re.split(r'\n\s*-{3,}\s*\n', texto)
    
    for aviso in avisos:
        # Verificar que sea JGA
        if not re.search(r'JUNTA\s+GENERAL', aviso, re.IGNORECASE):
            continue
        
        jga = {
            'tipo': 'JUNTA_ACCIONISTAS',
            'empresa': None,
            'fecha': None,
            'hora': None,
            'lugar': None,
            'agenda': None,
            'texto_completo': aviso[:500]
        }
        
        # Empresa
        match = re.search(PATRON_JGA['empresa'], aviso, re.IGNORECASE)
        if match:
            jga['empresa'] = match.group(1).strip()
        
        # Fecha
        match = re.search(PATRON_JGA['fecha_junta'], aviso, re.IGNORECASE)
        if match:
            dia, mes, año = match.groups()
            jga['fecha'] = f"{dia}/{mes}/{año}"
        
        # Hora
        match = re.search(PATRON_JGA['hora'], aviso)
        if match:
            jga['hora'] = match.group(1)
        
        # Lugar
        match = re.search(PATRON_JGA['lugar'], aviso, re.IGNORECASE)
        if match:
            jga['lugar'] = match.group(1).strip()
        
        if jga['empresa'] or jga['fecha']:
            convocatorias.append(jga)
    
    return convocatorias


# =========================================================================
# FUNCIÓN: EXTRAER DISOLUCIONES
# =========================================================================

def extraer_disoluciones(texto):
    """
    Extrae avisos de disolución y liquidación
    """
    
    disoluciones = []
    avisos = re.split(r'\n\s*-{3,}\s*\n', texto)
    
    for aviso in avisos:
        if not re.search(PATRON_DISOLUCION['titulo'], aviso, re.IGNORECASE):
            continue
        
        disolucion = {
            'tipo': 'DISOLUCION',
            'empresa': None,
            'ruc': None,
            'liquidador': None,
            'texto_completo': aviso[:500]
        }
        
        # Empresa
        match = re.search(PATRON_DISOLUCION['empresa'], aviso, re.IGNORECASE)
        if match:
            disolucion['empresa'] = match.group(1).strip()
        
        # RUC
        match = re.search(PATRON_DISOLUCION['ruc'], aviso)
        if match:
            disolucion['ruc'] = match.group(1)
        
        # Liquidador
        match = re.search(PATRON_DISOLUCION['liquidador'], aviso, re.IGNORECASE)
        if match:
            disolucion['liquidador'] = match.group(1).strip()
        
        if disolucion['empresa'] or disolucion['ruc']:
            disoluciones.append(disolucion)
    
    return disoluciones


# =========================================================================
# PROCESAR CORPUS
# =========================================================================

print(" Cargando corpus...")
ruta_corpus = CARPETA_DATOS / "corpus_limpio.json"

with open(ruta_corpus, 'r', encoding='utf-8') as f:
    corpus = json.load(f)

print(f" Cargados {len(corpus)} documentos\n")

print("="*70)
print(" EXTRAYENDO INFORMACIÓN ESTRUCTURADA")
print("="*70 + "\n")

# Estructura para guardar todo
base_datos = {
    'remates': [],
    'edictos': [],
    'licitaciones': [],
    'convocatorias_jga': [],
    'fusiones': [],
    'disoluciones': [],
    'requerimientos_sbs': []
}

# Estadísticas
stats = defaultdict(int)

# Procesar cada documento
for doc in tqdm(corpus, desc="Procesando documentos"):
    
    texto = doc['texto_limpio']
    metadata = {
        'archivo': doc['archivo'],
        'fecha_boletin': doc['fecha'],
        'año': doc['año'],
        'mes': doc['mes']
    }
    
    # Detectar secciones
    secciones = detectar_secciones(texto)
    
    # Extraer de cada sección
    
    # 1. REMATES
    for seccion in secciones['REMATES']:
        remates = extraer_remates(seccion)
        for remate in remates:
            remate.update(metadata)
            base_datos['remates'].append(remate)
            stats['remates'] += 1
    
    # 2. CONVOCATORIAS JGA
    for seccion in secciones['CONVOCATORIAS']:
        jgas = extraer_jga(seccion)
        for jga in jgas:
            jga.update(metadata)
            base_datos['convocatorias_jga'].append(jga)
            stats['convocatorias'] += 1
    
    # 3. DISOLUCIONES
    for seccion in secciones['DISOLUCIONES']:
        disoluciones = extraer_disoluciones(seccion)
        for disolucion in disoluciones:
            disolucion.update(metadata)
            base_datos['disoluciones'].append(disolucion)
            stats['disoluciones'] += 1

print(f"\n Extracción completada\n")

# =========================================================================
# GUARDAR BASE DE DATOS
# =========================================================================

print("="*70)
print(" GUARDANDO BASE DE DATOS")
print("="*70 + "\n")

# 1. JSON completo
print("📄 Base de datos completa (JSON)...", end=" ")
ruta_bd_json = CARPETA_EXTRAIDO / "base_datos_peruano.json"

with open(ruta_bd_json, 'w', encoding='utf-8') as f:
    json.dump(base_datos, f, ensure_ascii=False, indent=2)

print(f" ({ruta_bd_json.stat().st_size / (1024*1024):.1f} MB)")

# 2. CSV individuales por tipo
for tipo, datos in base_datos.items():
    if len(datos) > 0:
        print(f"📄 {tipo}.csv...", end=" ")
        ruta_csv = CARPETA_EXTRAIDO / f"{tipo}.csv"
        
        df = pd.DataFrame(datos)
        df.to_csv(ruta_csv, index=False, encoding='utf-8')
        
        print(f" ({len(datos)} registros)")

# 3. Estadísticas
print("\n📄 Estadísticas...", end=" ")
ruta_stats = CARPETA_EXTRAIDO / "estadisticas_extraccion.json"

estadisticas = {
    'total_documentos': len(corpus),
    'extracciones': dict(stats),
    'total_registros': sum(stats.values())
}

with open(ruta_stats, 'w', encoding='utf-8') as f:
    json.dump(estadisticas, f, ensure_ascii=False, indent=2)

print(f" ok ")

# =========================================================================
# RESUMEN
# =========================================================================

print(f"\n{'='*70}")
print(f" RESUMEN DE EXTRACCIÓN")
print(f"{'='*70}\n")

print(f" Documentos procesados: {len(corpus)}")
print(f" Total registros extraídos: {sum(stats.values())}\n")

print(f" Por tipo:")
for tipo, count in stats.items():
    print(f"   {tipo:20s}: {count:4d} registros")

print(f"\n{'='*70}")
print(f" ARCHIVOS GENERADOS")
print(f"{'='*70}\n")

print(f" {CARPETA_EXTRAIDO}/")
print(f"   ├── base_datos_peruano.json (base completa)")

for tipo in base_datos.keys():
    if len(base_datos[tipo]) > 0:
        print(f"   ├── {tipo}.csv")

print(f"   └── estadisticas_extraccion.json")

print(f"\n{'='*70}")
print(f" EXTRACCIÓN COMPLETADA")
print(f"{'='*70}\n")

print(f" Próximos pasos:")
print(f"   1. Revisar CSVs individuales")
print(f"   2. Ajustar patrones de extracción")
print(f"   3. Crear dashboard/visualizaciones")
print(f"   4. Análisis de tendencias\n")
"""


 EXTRACTOR ESTRUCTURADO - DIARIO EL PERUANO

 Cargando corpus...
 Cargados 243 documentos

 EXTRAYENDO INFORMACIÓN ESTRUCTURADA



Procesando documentos: 100%|██████████| 243/243 [00:00<00:00, 5796.99it/s]


 Extracción completada

 GUARDANDO BASE DE DATOS

📄 Base de datos completa (JSON)...  (0.0 MB)

📄 Estadísticas...  ok 

 RESUMEN DE EXTRACCIÓN

 Documentos procesados: 243
 Total registros extraídos: 0

 Por tipo:

 ARCHIVOS GENERADOS

 informacion_extraida/
   ├── base_datos_peruano.json (base completa)
   └── estadisticas_extraccion.json

 EXTRACCIÓN COMPLETADA

 Próximos pasos:
   1. Revisar CSVs individuales
   2. Ajustar patrones de extracción
   3. Crear dashboard/visualizaciones
   4. Análisis de tendencias



In [11]:
# =========================================================================
# EXTRACTOR ESTRUCTURADO SIMPLIFICADO
# =========================================================================
# Busca directamente en el texto SIN depender de deteccion de secciones
# =========================================================================

import re
import json
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict
import pandas as pd

print("\n" + "="*70)
print(" EXTRACTOR ESTRUCTURADO SIMPLIFICADO")
print("="*70 + "\n")

# =========================================================================
# CONFIGURACION
# =========================================================================

CARPETA_DATOS = Path("datos_limpios")
CARPETA_EXTRAIDO = Path("informacion_extraida")
CARPETA_EXTRAIDO.mkdir(exist_ok=True)

# =========================================================================
# FUNCIONES DE EXTRACCION SIMPLIFICADAS
# =========================================================================

def extraer_juntas(texto):
    """Busca juntas de accionistas en todo el texto"""
    
    juntas = []
    
    # Patron: buscar bloques con "JUNTA GENERAL"
    patron_junta = r'JUNTA\s+GENERAL.*?(?=JUNTA\s+GENERAL|$)'
    
    matches = re.finditer(patron_junta, texto, re.IGNORECASE | re.DOTALL)
    
    for match in matches:
        bloque = match.group(0)[:1000]  # Primeros 1000 caracteres
        
        # Extraer empresa
        empresa = None
        match_empresa = re.search(r'(?:DE|EN)\s+([A-ZÁÉÍÓÚÑ\s\.]+?S\.?A\.?C?\.?)', bloque)
        if match_empresa:
            empresa = match_empresa.group(1).strip()
        
        # Extraer fecha
        fecha = None
        match_fecha = re.search(r'(\d{1,2})\s+DE\s+(\w+)\s+(?:DE|DEL)?\s*(\d{4})', bloque, re.IGNORECASE)
        if match_fecha:
            dia, mes, anio = match_fecha.groups()
            fecha = f"{dia}/{mes}/{anio}"
        
        if empresa or fecha or len(bloque) > 100:
            juntas.append({
                'tipo': 'JUNTA_ACCIONISTAS',
                'empresa': empresa,
                'fecha': fecha,
                'texto_completo': bloque[:600]
            })
    
    return juntas


def extraer_disoluciones(texto):
    """Busca disoluciones en todo el texto"""
    
    disoluciones = []
    
    # Patron: buscar bloques con "DISOLUCION"
    patron_disolucion = r'DISOLUCI[OÓ]N.*?(?=DISOLUCI[OÓ]N|JUNTA\s+GENERAL|$)'
    
    matches = re.finditer(patron_disolucion, texto, re.IGNORECASE | re.DOTALL)
    
    for match in matches:
        bloque = match.group(0)[:1000]
        
        # Extraer empresa
        empresa = None
        match_empresa = re.search(r'(?:EMPRESA|SOCIEDAD|RAZÓN SOCIAL)[:\s]+([A-ZÁÉÍÓÚÑ\s\.]+?S\.?A\.?C?\.?)', bloque, re.IGNORECASE)
        if not match_empresa:
            match_empresa = re.search(r'([A-ZÁÉÍÓÚÑ\s\.]+?S\.?A\.?C?\.?)', bloque)
        if match_empresa:
            empresa = match_empresa.group(1).strip()
        
        # Extraer RUC
        ruc = None
        match_ruc = re.search(r'RUC[:\s]*N?[°º]?\s*(\d{11})', bloque, re.IGNORECASE)
        if match_ruc:
            ruc = match_ruc.group(1)
        
        if empresa or ruc or len(bloque) > 100:
            disoluciones.append({
                'tipo': 'DISOLUCION',
                'empresa': empresa,
                'ruc': ruc,
                'texto_completo': bloque[:600]
            })
    
    return disoluciones


def extraer_remates(texto):
    """Busca remates en todo el texto"""
    
    remates = []
    
    # Patron: buscar bloques con "REMATE" o "SUBASTA"
    patron_remate = r'(?:REMATE|SUBASTA).*?(?=REMATE|SUBASTA|JUNTA\s+GENERAL|DISOLUCI[OÓ]N|$)'
    
    matches = re.finditer(patron_remate, texto, re.IGNORECASE | re.DOTALL)
    
    for match in matches:
        bloque = match.group(0)[:1000]
        
        # Extraer ubicacion
        ubicacion = None
        match_ubicacion = re.search(r'UBICACI[OÓ]N[:\s]+([^\n]+)', bloque, re.IGNORECASE)
        if match_ubicacion:
            ubicacion = match_ubicacion.group(1).strip()
        
        # Extraer base
        base = None
        match_base = re.search(r'BASE[:\s]*S/\.?\s*([\d,]+\.?\d*)', bloque, re.IGNORECASE)
        if match_base:
            base = match_base.group(1).replace(',', '')
        
        # Extraer fecha
        fecha = None
        match_fecha = re.search(r'FECHA[:\s]*(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})', bloque, re.IGNORECASE)
        if match_fecha:
            fecha = match_fecha.group(1)
        
        if ubicacion or base or fecha or len(bloque) > 100:
            remates.append({
                'tipo': 'REMATE',
                'ubicacion': ubicacion,
                'base': base,
                'fecha': fecha,
                'texto_completo': bloque[:600]
            })
    
    return remates


# =========================================================================
# CARGAR Y PROCESAR CORPUS
# =========================================================================

print(" Cargando corpus...")
ruta_corpus = CARPETA_DATOS / "corpus_limpio.json"

with open(ruta_corpus, 'r', encoding='utf-8') as f:
    corpus = json.load(f)

print(f" Cargados {len(corpus)} documentos\n")

print("="*70)
print(" EXTRAYENDO AVISOS")
print("="*70 + "\n")

# Estructura para guardar
base_datos = {
    'remates': [],
    'convocatorias_jga': [],
    'disoluciones': []
}

stats = defaultdict(int)

# Procesar cada documento
for doc in tqdm(corpus, desc="Procesando"):
    
    texto = doc['texto_limpio']
    metadata = {
        'archivo': doc['archivo'],
        'fecha': doc['fecha'],
        'año': doc.get('año'),
        'mes': doc.get('mes')
    }
    
    # Extraer juntas
    juntas = extraer_juntas(texto)
    for junta in juntas:
        junta.update(metadata)
        base_datos['convocatorias_jga'].append(junta)
        stats['juntas'] += 1
    
    # Extraer disoluciones
    disoluciones = extraer_disoluciones(texto)
    for disolucion in disoluciones:
        disolucion.update(metadata)
        base_datos['disoluciones'].append(disolucion)
        stats['disoluciones'] += 1
    
    # Extraer remates
    remates = extraer_remates(texto)
    for remate in remates:
        remate.update(metadata)
        base_datos['remates'].append(remate)
        stats['remates'] += 1

print(f"\n Extraccion completada\n")

# =========================================================================
# GUARDAR CSVs
# =========================================================================

print("="*70)
print(" GUARDANDO ARCHIVOS")
print("="*70 + "\n")

for tipo, datos in base_datos.items():
    if len(datos) > 0:
        print(f" {tipo}.csv...", end=" ")
        ruta_csv = CARPETA_EXTRAIDO / f"{tipo}.csv"
        
        df = pd.DataFrame(datos)
        df.to_csv(ruta_csv, index=False, encoding='utf-8')
        
        print(f"OK ({len(datos)} registros)")

# =========================================================================
# RESUMEN
# =========================================================================

print(f"\n{'='*70}")
print(f" RESUMEN")
print(f"{'='*70}\n")

print(f" Documentos procesados: {len(corpus)}")
print(f" Total registros: {sum(stats.values())}\n")

print(f" Por tipo:")
for tipo, count in stats.items():
    print(f"   {tipo:20s}: {count:4d}")

print(f"\n{'='*70}")
print(f" ARCHIVOS EN: {CARPETA_EXTRAIDO}/")
print(f"{'='*70}\n")

for tipo in base_datos.keys():
    if len(base_datos[tipo]) > 0:
        print(f"   {tipo}.csv")

print(f"\n{'='*70}")
print(f" COMPLETADO")
print(f"{'='*70}\n")

print(" PROXIMO PASO:")
print("   Ejecutar celda de CONSOLIDACION")


 EXTRACTOR ESTRUCTURADO SIMPLIFICADO

 Cargando corpus...
 Cargados 243 documentos

 EXTRAYENDO AVISOS



Procesando: 100%|██████████| 243/243 [00:12<00:00, 20.16it/s]



 Extraccion completada

 GUARDANDO ARCHIVOS

OK (9515 registros)
 convocatorias_jga.csv... OK (3804 registros)
 disoluciones.csv... OK (3689 registros)

 RESUMEN

 Documentos procesados: 243
 Total registros: 17008

 Por tipo:
   juntas              : 3804
   disoluciones        : 3689
   remates             : 9515

 ARCHIVOS EN: informacion_extraida/

   remates.csv
   convocatorias_jga.csv
   disoluciones.csv

 COMPLETADO

 PROXIMO PASO:
   Ejecutar celda de CONSOLIDACION


In [12]:
# =========================================================================
# CELDA QUE FALTA: CONSOLIDAR TODO EN base_datos_completa.csv
# =========================================================================
# Esta celda junta remates, juntas, disoluciones en UN SOLO CSV
# EJECUTAR DESPUES de extraer entidades o reestructurar CSVs
# =========================================================================

import pandas as pd
from pathlib import Path

print("\n" + "="*70)
print(" CONSOLIDANDO TODO EN base_datos_completa.csv")
print("="*70 + "\n")

# =========================================================================
# PASO 1: BUSCAR Y CARGAR TODOS LOS CSVs
# =========================================================================

# Buscar en diferentes carpetas
carpetas = [
    Path("informacion_extraida"),
    Path("datos_estructurados"),
    Path("entidades_mejorado"),
]

todos_los_datos = []

for carpeta in carpetas:
    if not carpeta.exists():
        continue
    
    print(f" Buscando en: {carpeta}/\n")
    
    # Buscar CSVs
    csvs = list(carpeta.glob("*.csv"))
    
    for csv_path in csvs:
        try:
            df = pd.read_csv(csv_path, encoding='utf-8')
            
            # Agregar tipo si no existe
            if 'tipo' not in df.columns:
                # Inferir tipo del nombre del archivo
                nombre = csv_path.stem.lower()
                
                if 'junta' in nombre or 'convocatoria' in nombre:
                    df['tipo'] = 'JUNTA_ACCIONISTAS'
                elif 'disolucion' in nombre:
                    df['tipo'] = 'DISOLUCION'
                elif 'remate' in nombre:
                    df['tipo'] = 'REMATE'
                else:
                    df['tipo'] = 'AVISO'
            
            # Agregar archivo si no existe
            if 'archivo' not in df.columns and 'fecha_boletin' in df.columns:
                # Crear nombre de archivo desde fecha
                df['archivo'] = 'boletin_' + df['fecha_boletin'].astype(str).str.replace('-', '') + '.pdf'
            
            # Agregar texto_completo si no existe
            if 'texto_completo' not in df.columns:
                if 'texto' in df.columns:
                    df['texto_completo'] = df['texto']
                else:
                    df['texto_completo'] = ""
            
            # Solo agregar si tiene registros
            if len(df) > 0:
                print(f"    {csv_path.name}: {len(df)} registros ({df['tipo'].iloc[0] if 'tipo' in df.columns else 'AVISO'})")
                todos_los_datos.append(df)
        
        except Exception as e:
            print(f"    Error: {csv_path.name} - {e}")
    
    print()

# =========================================================================
# PASO 2: CONSOLIDAR TODO
# =========================================================================

if len(todos_los_datos) == 0:
    print(" No se encontraron CSVs para consolidar")
else:
    print("="*70)
    print(" CONSOLIDANDO...")
    print("="*70 + "\n")
    
    # Concatenar todos los DataFrames
    df_consolidado = pd.concat(todos_los_datos, ignore_index=True)
    
    print(f" Total registros: {len(df_consolidado)}")
    
    # Verificar columnas esenciales
    columnas_esenciales = ['archivo', 'tipo', 'texto_completo']
    
    for col in columnas_esenciales:
        if col not in df_consolidado.columns:
            print(f"    Falta columna: {col}, agregando vacía")
            df_consolidado[col] = ""
    
    # =====================================================================
    # PASO 3: ESTADISTICAS
    # =====================================================================
    
    print(f"\n ESTADISTICAS POR TIPO:\n")
    
    conteo = df_consolidado['tipo'].value_counts()
    for tipo, count in conteo.items():
        print(f"   {tipo:30} {count:4d} registros")
    
    # =====================================================================
    # PASO 4: GUARDAR
    # =====================================================================
    
    print(f"\n{'='*70}")
    print(" GUARDANDO")
    print("="*70 + "\n")
    
    carpeta_salida = Path("base_datos_final")
    carpeta_salida.mkdir(exist_ok=True)
    
    # CSV
    ruta_csv = carpeta_salida / "base_datos_completa.csv"
    df_consolidado.to_csv(ruta_csv, index=False, encoding='utf-8')
    
    tamano_mb = ruta_csv.stat().st_size / (1024*1024)
    
    print(f" {ruta_csv}")
    print(f"   Registros: {len(df_consolidado):,}")
    print(f"   Columnas: {len(df_consolidado.columns)}")
    print(f"   Tamaño: {tamano_mb:.1f} MB")
    
    # Mostrar columnas
    print(f"\n COLUMNAS:")
    for col in df_consolidado.columns:
        print(f"   • {col}")
    
    print(f"\n{'='*70}")
    print(" CONSOLIDACION COMPLETADA")
    print("="*70 + "\n")
    
    print("PROXIMO PASO:")
    print("   streamlit run dashboard_busqueda_simple4.py")

print("\n" + "="*70 + "\n")


 CONSOLIDANDO TODO EN base_datos_completa.csv

 Buscando en: informacion_extraida/

    convocatorias_jga.csv: 3804 registros (JUNTA_ACCIONISTAS)
    disoluciones.csv: 3689 registros (DISOLUCION)
    remates.csv: 9515 registros (REMATE)

 Buscando en: entidades_mejorado/

    entidades_mejorado.csv: 3043025 registros (MISC)

 CONSOLIDANDO...

 Total registros: 3060033

 ESTADISTICAS POR TIPO:

   MISC                           1284566 registros
   LOC                            796607 registros
   ORG                            729075 registros
   PER                            180463 registros
   MONTO_SOLES                    22685 registros
   LEY                            22087 registros
   REMATE                         9515 registros
   JUNTA_ACCIONISTAS              3804 registros
   DISOLUCION                     3689 registros
   DECRETO_SUPREMO                3111 registros
   MONTO_DOLARES                  1956 registros
   RUC                            1935 registros
   

## LIMPIEZA DE HTML DE LA BASE DE DATOS

In [13]:
# =========================================================================
# LIMPIAR HTML DE LA BASE DE DATOS (VERSIÓN CORREGIDA)
# =========================================================================

import pandas as pd
from pathlib import Path
import re
from bs4 import BeautifulSoup

print("\n" + "="*70)
print(" LIMPIANDO HTML DE LA BASE DE DATOS")
print("="*70 + "\n")

# =========================================================================
# CARGAR BASE DE DATOS
# =========================================================================

ruta = Path("base_datos_final/base_datos_completa.csv")

if not ruta.exists():
    print(" ERROR: El archivo no existe")
    print(f"   Buscado en: {ruta}")
    print("\n  SOLUCIÓN:")
    print("   1. Ejecuta primero la celda 'CREAR base_datos_completa.csv'")
else:
    df = pd.read_csv(ruta, encoding='utf-8')
    print(f" Cargados {len(df)} registros\n")
    
    # Mostrar columnas disponibles
    print(" Columnas disponibles:")
    for col in df.columns:
        print(f"   • {col}")
    
    # =====================================================================
    # FUNCIÓN PARA LIMPIAR HTML
    # =====================================================================
    
    def limpiar_html(texto):
        """
        Elimina todo el HTML y deja solo texto plano
        """
        if pd.isna(texto):
            return ""
        
        texto = str(texto)
        
        # Usar BeautifulSoup para quitar HTML
        try:
            soup = BeautifulSoup(texto, 'html.parser')
            texto_limpio = soup.get_text(separator=' ')
        except:
            # Si falla BeautifulSoup, usar regex simple
            texto_limpio = re.sub(r'<[^>]+>', '', texto)
        
        # Limpiar espacios múltiples
        texto_limpio = re.sub(r'\s+', ' ', texto_limpio)
        
        return texto_limpio.strip()
    
    # =====================================================================
    # IDENTIFICAR COLUMNAS DE TEXTO PARA LIMPIAR
    # =====================================================================
    
    print("\n Identificando columnas con posible HTML...\n")
    
    columnas_limpiar = []
    
    # Buscar columnas que probablemente contengan texto
    posibles_columnas = ['texto', 'texto_completo', 'texto_formateado', 'contenido', 'descripcion']
    
    for col in posibles_columnas:
        if col in df.columns:
            columnas_limpiar.append(col)
            print(f"    Limpiando: {col}")
    
    # Si no encontró columnas conocidas, buscar cualquier columna con 'texto' en el nombre
    if not columnas_limpiar:
        columnas_limpiar = [col for col in df.columns if 'texto' in col.lower()]
        for col in columnas_limpiar:
            print(f"    Limpiando: {col}")
    
    if not columnas_limpiar:
        print("     No se encontraron columnas de texto para limpiar")
        print("   Se usará la primera columna que parezca texto")
        # Usar la primera columna que tenga strings
        for col in df.columns:
            if df[col].dtype == 'object':
                columnas_limpiar = [col]
                print(f"    Usando: {col}")
                break
    
    # =====================================================================
    # LIMPIAR HTML
    # =====================================================================
    
    if columnas_limpiar:
        print(f"\n Limpiando HTML de {len(columnas_limpiar)} columna(s)...\n")
        
        for col in columnas_limpiar:
            print(f"   Procesando: {col}...")
            df[col] = df[col].apply(limpiar_html)
            print(f"    {col} limpiado")
        
        # =====================================================================
        # GUARDAR BASE DE DATOS LIMPIA
        # =====================================================================
        
        print("\n Guardando base de datos limpia...\n")
        
        # CSV
        ruta_csv = Path("base_datos_final/base_datos_completa.csv")
        df.to_csv(ruta_csv, index=False, encoding='utf-8')
        print(f"    CSV guardado: {ruta_csv}")
        print(f"      Tamaño: {ruta_csv.stat().st_size / 1024:.1f} KB")
        
        # JSON
        ruta_json = Path("base_datos_final/base_datos_completa.json")
        df.to_json(ruta_json, orient='records', force_ascii=False, indent=2)
        print(f"    JSON guardado: {ruta_json}")
        
        # =====================================================================
        # RESUMEN
        # =====================================================================
        
        print(f"\n{'='*70}")
        print(" BASE DE DATOS LIMPIADA")
        print("="*70 + "\n")
        
        print(" MUESTRA DE TEXTO LIMPIO:")
        print("-" * 70)
        
        # Mostrar una muestra de la primera columna limpiada
        if columnas_limpiar:
            col_muestra = columnas_limpiar[0]
            if len(df) > 0:
                muestra = str(df.iloc[0][col_muestra])[:300]
                print(muestra)
                if len(muestra) >= 300:
                    print("...")
        
        print("-" * 70 + "\n")
        
        print(" PRÓXIMO PASO:")
        print("   El dashboard ahora cargará datos sin HTML")
        print("   Ejecuta: streamlit run dashboard_busqueda_simple4.py")
        print()

print("="*70 + "\n")


 LIMPIANDO HTML DE LA BASE DE DATOS



C:\Users\Carlos\AppData\Local\Temp\ipykernel_11424\3325230734.py:26: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, encoding='utf-8')


 Cargados 3060033 registros

 Columnas disponibles:
   • tipo
   • empresa
   • fecha
   • texto_completo
   • archivo
   • año
   • mes
   • ruc
   • ubicacion
   • base
   • texto
   • inicio
   • fin
   • texto_normalizado

 Identificando columnas con posible HTML...

    Limpiando: texto
    Limpiando: texto_completo

 Limpiando HTML de 2 columna(s)...

   Procesando: texto...


C:\Users\Carlos\AppData\Local\Temp\ipykernel_11424\3325230734.py:49: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(texto, 'html.parser')
C:\Users\Carlos\AppData\Local\Temp\ipykernel_11424\3325230734.py:49: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(texto, 'html.parser')


    texto limpiado
   Procesando: texto_completo...
    texto_completo limpiado

 Guardando base de datos limpia...

    CSV guardado: base_datos_final\base_datos_completa.csv
      Tamaño: 354555.2 KB
    JSON guardado: base_datos_final\base_datos_completa.json

 BASE DE DATOS LIMPIADA

 MUESTRA DE TEXTO LIMPIO:
----------------------------------------------------------------------

----------------------------------------------------------------------

 PRÓXIMO PASO:
   El dashboard ahora cargará datos sin HTML
   Ejecuta: streamlit run dashboard_busqueda_simple4.py




## STREAMLIT

In [14]:
codigo_mejorado_v3 = """import streamlit as st
import pandas as pd
from pathlib import Path
import math

st.set_page_config(page_title="Buscador El Peruano", page_icon="🔍", layout="wide")

st.markdown('''
<style>
    .titulo { font-size: 2rem; font-weight: bold; color: #1f77b4; text-align: center; }
    .card { background: #f8f9fa; padding: 1.5rem; border-radius: 0.5rem; border-left: 4px solid #1f77b4; margin: 1rem 0; }
    .badge { display: inline-block; padding: 0.25rem 0.5rem; border-radius: 0.25rem; font-size: 0.75rem; font-weight: bold; margin-right: 0.5rem; }
    .badge-decreto { background-color: #cfe2ff; color: #084298; }
    .badge-ley { background-color: #e2d9f3; color: #432874; }
    .badge-resolucion { background-color: #f8d7da; color: #721c24; }
    .badge-junta { background-color: #d1ecf1; color: #0c5460; }
    .badge-disolucion { background-color: #f8d7da; color: #721c24; }
    .badge-remate { background-color: #d4edda; color: #155724; }
    .texto-preview { 
        font-family: Arial, sans-serif; 
        font-size: 0.95rem; 
        line-height: 1.6; 
        color: #333;
        white-space: pre-wrap;
        margin-top: 1rem;
        background: white;
        padding: 1rem;
        border-radius: 0.5rem;
    }
</style>
''', unsafe_allow_html=True)

def cortar_texto(texto, max_chars=800):
    if len(texto) <= max_chars:
        return texto, False
    
    texto_corto = texto[:max_chars]
    ultimo_espacio = texto_corto.rfind(' ')
    if ultimo_espacio > max_chars * 0.7:
        return texto[:ultimo_espacio] + '...', True
    
    return texto[:max_chars] + '...', True

# CARGAR DATOS
@st.cache_data(show_spinner="Cargando datos...")
def cargar_datos():
    ruta = Path("base_datos_final/base_datos_completa.csv")
    
    if not ruta.exists():
        return None
    
    df = pd.read_csv(ruta, encoding='utf-8', low_memory=False)
    return df

df = cargar_datos()

if df is None:
    st.error(" No se encontró base_datos_completa.csv")
    st.info("Ejecuta la celda de CONSOLIDACIÓN en tu notebook primero")
    st.stop()

# Filtrar tipos relevantes
TIPOS_RELEVANTES = [
    'DECRETO_SUPREMO', 'RESOLUCION_SUPREMA', 'RESOLUCION_MINISTERIAL',
    'LEY', 'JUNTA_ACCIONISTAS', 'DISOLUCION', 'REMATE', 'AVISO'
]

if 'tipo' in df.columns:
    df = df[df['tipo'].isin(TIPOS_RELEVANTES)]

st.markdown('<h1 class="titulo"> Buscador Diario El Peruano</h1>', unsafe_allow_html=True)
st.markdown(f"<p style='text-align: center; color: gray;'>Base de datos con {len(df)} avisos legales</p>", unsafe_allow_html=True)

# Filtros
st.sidebar.markdown("##  Filtros")
busqueda = st.sidebar.text_input("Buscar en texto", placeholder="Ej: SUNAT, remate...")

tipos_disponibles = ['Todos']
if 'tipo' in df.columns:
    tipos_disponibles += sorted(df['tipo'].unique().tolist())
tipo = st.sidebar.selectbox("Tipo de documento", tipos_disponibles)

empresa = ""
if 'empresa' in df.columns:
    empresa = st.sidebar.text_input("Empresa", placeholder="Ej: TRANSPORTES...")

ruc = ""
if 'ruc' in df.columns:
    ruc = st.sidebar.text_input("RUC", placeholder="20100154057")

orden = st.sidebar.radio("Ordenar por fecha", ["Más recientes", "Más antiguos"])

# Aplicar filtros
df_filtrado = df.copy()

# Filtro por tipo
if tipo != 'Todos' and 'tipo' in df.columns:
    df_filtrado = df_filtrado[df_filtrado['tipo'] == tipo]

# Filtro por empresa
if empresa and 'empresa' in df.columns:
    df_filtrado = df_filtrado[df_filtrado['empresa'].str.contains(empresa, case=False, na=False)]

# Filtro por RUC
if ruc and 'ruc' in df.columns:
    df_filtrado = df_filtrado[df_filtrado['ruc'].astype(str).str.contains(ruc, na=False)]

# Filtro por búsqueda en texto
if busqueda:
    # Buscar en texto_completo o texto
    col_texto = None
    for col in ['texto_completo', 'texto', 'contenido']:
        if col in df_filtrado.columns:
            col_texto = col
            break
    
    if col_texto:
        df_filtrado = df_filtrado[df_filtrado[col_texto].str.contains(busqueda, case=False, na=False)]

# Ordenar por fecha
col_fecha = None
for col in ['fecha_boletin', 'fecha', 'fecha_publicacion']:
    if col in df_filtrado.columns:
        col_fecha = col
        break

if col_fecha:
    if orden == "Más recientes":
        df_filtrado = df_filtrado.sort_values(col_fecha, ascending=False)
    else:
        df_filtrado = df_filtrado.sort_values(col_fecha, ascending=True)

# Métricas
st.markdown("---")
col1, col2, col3, col4 = st.columns(4)
col1.metric(" Resultados", len(df_filtrado))

if 'tipo' in df_filtrado.columns:
    col2.metric(" Juntas", len(df_filtrado[df_filtrado['tipo'] == 'JUNTA_ACCIONISTAS']))
    col3.metric(" Disoluciones", len(df_filtrado[df_filtrado['tipo'] == 'DISOLUCION']))
    col4.metric(" Remates", len(df_filtrado[df_filtrado['tipo'] == 'REMATE']))

st.markdown("---")

# Paginación
RESULTADOS_POR_PAGINA = 10
total_resultados = len(df_filtrado)
total_paginas = math.ceil(total_resultados / RESULTADOS_POR_PAGINA) if total_resultados > 0 else 1

if 'pagina' not in st.session_state:
    st.session_state.pagina = 1

# Reset página si cambian filtros
filtros_actuales = f"{busqueda}_{tipo}_{empresa}_{ruc}_{orden}"
if 'filtros_anteriores' not in st.session_state:
    st.session_state.filtros_anteriores = filtros_actuales
elif st.session_state.filtros_anteriores != filtros_actuales:
    st.session_state.pagina = 1
    st.session_state.filtros_anteriores = filtros_actuales

if total_resultados > 0:
    # Navegación
    col1, col2, col3 = st.columns([1, 2, 1])
    
    with col1:
        if st.session_state.pagina > 1:
            if st.button(" Anterior"):
                st.session_state.pagina -= 1
                st.rerun()
    
    with col2:
        pagina_actual = st.selectbox(
            "Página",
            options=list(range(1, total_paginas + 1)),
            index=st.session_state.pagina - 1,
            key="selector_pagina"
        )
        if pagina_actual != st.session_state.pagina:
            st.session_state.pagina = pagina_actual
            st.rerun()
    
    with col3:
        if st.session_state.pagina < total_paginas:
            if st.button("Siguiente "):
                st.session_state.pagina += 1
                st.rerun()
    
    inicio = (st.session_state.pagina - 1) * RESULTADOS_POR_PAGINA
    fin = min(inicio + RESULTADOS_POR_PAGINA, total_resultados)
    
    st.markdown(f"### Mostrando {inicio + 1} - {fin} de {total_resultados} resultados")
    
    # Mostrar resultados
    for idx, row in df_filtrado.iloc[inicio:fin].iterrows():
        tipo_doc = row['tipo'] if 'tipo' in df_filtrado.columns and pd.notna(row['tipo']) else 'AVISO'
        
        # Badge con color
        badge_map = {
            'DECRETO_SUPREMO': ('DECRETO SUPREMO', 'badge-decreto'),
            'RESOLUCION_SUPREMA': ('RESOLUCIÓN SUPREMA', 'badge-resolucion'),
            'LEY': ('LEY', 'badge-ley'),
            'JUNTA_ACCIONISTAS': ('JUNTA', 'badge-junta'),
            'DISOLUCION': ('DISOLUCIÓN', 'badge-disolucion'),
            'REMATE': ('REMATE', 'badge-remate'),
        }
        
        badge_texto, badge_clase = badge_map.get(tipo_doc, (tipo_doc, 'badge'))
        badge = f'<span class="badge {badge_clase}">{badge_texto}</span>'
        
        # Metadata
        metadata_parts = []
        if col_fecha and pd.notna(row[col_fecha]):
            metadata_parts.append(f" Fecha: {row[col_fecha]}")
        if 'empresa' in df_filtrado.columns and pd.notna(row['empresa']) and str(row['empresa']).strip():
            metadata_parts.append(f" Empresa: {row['empresa']}")
        if 'ruc' in df_filtrado.columns and pd.notna(row['ruc']) and str(row['ruc']).strip():
            metadata_parts.append(f" RUC: {row['ruc']}")
        
        metadata = " | ".join(metadata_parts) if metadata_parts else "Sin metadata"
        
        st.markdown(
            f'<div class="card">{badge}<br><small>{metadata}</small></div>',
            unsafe_allow_html=True
        )
        
        # Mostrar texto
        col_texto = None
        for col in ['texto_completo', 'texto', 'contenido']:
            if col in df_filtrado.columns:
                col_texto = col
                break
        
        if col_texto and pd.notna(row[col_texto]):
            texto = str(row[col_texto])
            
            if len(texto.strip()) > 10:
                texto_preview, necesita_expansion = cortar_texto(texto, max_chars=800)
                st.markdown(f'<div class="texto-preview">{texto_preview}</div>', unsafe_allow_html=True)
                
                if necesita_expansion:
                    with st.expander("📄 Ver texto completo"):
                        st.text(texto)
            else:
                st.info("Sin texto disponible")
        else:
            st.info("Sin texto disponible")
        
        st.markdown("<br>", unsafe_allow_html=True)
    
    st.markdown("---")
    st.markdown(
        f"<p style='text-align: center;'>Página {st.session_state.pagina} de {total_paginas}</p>",
        unsafe_allow_html=True
    )
else:
    st.warning(" No se encontraron resultados con los filtros seleccionados")

# Descarga
if total_resultados > 0:
    st.markdown("---")
    
    if total_resultados <= 5000:
        csv = df_filtrado.to_csv(index=False, encoding='utf-8')
        st.download_button(
            " Descargar resultados (CSV)", 
            csv, 
            "resultados_peruano.csv", 
            "text/csv"
        )
    else:
        st.warning(f" Demasiados resultados ({total_resultados:,}). Usa filtros para reducir a menos de 5,000.")

st.markdown("---")
st.markdown("<p style='text-align: center; color: gray;'> Buscador Diario El Peruano</p>", unsafe_allow_html=True)
"""

with open('dashboard_busqueda_simple4.py', 'w', encoding='utf-8') as f:
    f.write(codigo_mejorado_v3)

print(" Dashboard MEJORADO V3 creado")
print("\n MEJORAS:")
print("    Aumentado a 1500 caracteres (antes 1000)")
print("    Busca puntos en los últimos 200 caracteres")
print("    Busca punto + espacio/salto de línea")
print("    No corta palabras a la mitad")
print("    Algoritmo de corte mejorado")
print("\n Ahora en la terminal:")
print("   Ctrl+C")
print("   streamlit run dashboard_busqueda_simple4.py")

 Dashboard MEJORADO V3 creado

 MEJORAS:
    Aumentado a 1500 caracteres (antes 1000)
    Busca puntos en los últimos 200 caracteres
    Busca punto + espacio/salto de línea
    No corta palabras a la mitad
    Algoritmo de corte mejorado

 Ahora en la terminal:
   Ctrl+C
   streamlit run dashboard_busqueda_simple4.py


In [10]:
# DIAGNOSTICO: Ver contenido de corpus_limpio.json
import json
from pathlib import Path

ruta_corpus = Path("datos_limpios/corpus_limpio.json")

with open(ruta_corpus, 'r', encoding='utf-8') as f:
    corpus = json.load(f)

print(f"Total documentos: {len(corpus)}")
print(f"\nPrimer documento:")
print(f"Archivo: {corpus[0]['archivo']}")
print(f"Fecha: {corpus[0]['fecha']}")
print(f"Texto (primeros 500 chars):")
print(corpus[0]['texto_limpio'][:500])

# Buscar palabras clave en el primer documento
texto = corpus[0]['texto_limpio'].upper()
print(f"\n¿Contiene 'JUNTA GENERAL'? {('JUNTA GENERAL' in texto)}")
print(f"¿Contiene 'DISOLUCION'? {('DISOLUCION' in texto)}")
print(f"¿Contiene 'REMATE'? {('REMATE' in texto)}")

Total documentos: 243

Primer documento:
Archivo: boletin_20250401.pdf
Fecha: 2025-04-01
Texto (primeros 500 chars):
Gerente de Publicaciones Oficiales: MARTES 1 RICARDO MONTERO REYES DE ABRIL DE 2025 “AÑO DE LA RECUPERACIÓN Y CONSOLIDACIÓN DE LA ECONOMÍA PERUANA” FUNDADO EL 22 DE OCTUBRE DE 1825 POR EL LIBERTADOR SIMÓN BOLIVAR BOLETÍN OFICIAL SUMARIO AVISOS DE CURSO LEGAL ADMINISTRACIÓN DE JUSTICIA B. Edictos Clasificados 13 A. Avisos Diversos 1 A. Edictos Judiciales 12 • Inscripción y Rectificación de Partidas • Patrimonio B. Edictos Matrimoniales 11 • Juzgados Especializados • Juzgados de Paz Letrados Famil

¿Contiene 'JUNTA GENERAL'? True
¿Contiene 'DISOLUCION'? True
¿Contiene 'REMATE'? True


## 📊 PASO 7: Análisis Exploratorio de Datos

In [ ]:
# Cargar datos
try:
    df = pd.read_csv('entidades_extraidas.csv')
    df['fecha'] = pd.to_datetime(df['fecha'], format='%Y%m%d')
    print(f" Datos cargados: {len(df)} entidades")
except FileNotFoundError:
    print(" Archivo no encontrado. Ejecuta primero la extracción de entidades.")

In [ ]:
# Resumen estadístico
print("\n" + "="*70)
print(" RESUMEN GENERAL")
print("="*70 + "\n")

print(f" Período: {df['fecha'].min().date()} a {df['fecha'].max().date()}")
print(f" Total de entidades: {len(df):,}")
print(f" Días con datos: {df['fecha'].nunique()}")
print("\n Entidades por tipo:")
print(df['tipo'].value_counts())

print("\n Top 10 Instituciones:")
if len(df[df['tipo']=='ORG']) > 0:
    print(df[df['tipo']=='ORG']['texto'].value_counts().head(10))
else:
    print("   No se detectaron instituciones")

## 📈 PASO 8: Visualizaciones

In [ ]:
# Gráfico 1: Tendencia temporal general
por_fecha = df.groupby('fecha').size().reset_index(name='cantidad')

plt.figure(figsize=(14, 6))
plt.plot(por_fecha['fecha'], por_fecha['cantidad'], marker='o', linewidth=2, markersize=4)
plt.title('Evolución Temporal de Publicaciones - Diario El Peruano', fontsize=16, fontweight='bold')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Número de Entidades Detectadas', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(CARPETA_GRAFICOS / 'tendencia_general.png', dpi=300, bbox_inches='tight')
plt.show()

print(" Gráfico guardado: tendencia_general.png")

In [ ]:
# Gráfico 2: Top instituciones
if len(df[df['tipo']=='ORG']) > 0:
    orgs = df[df['tipo']=='ORG']
    top_orgs = orgs['texto'].value_counts().head(15)
    
    plt.figure(figsize=(12, 8))
    top_orgs.plot(kind='barh', color='steelblue')
    plt.title('Top 15 Instituciones Más Mencionadas', fontsize=16, fontweight='bold')
    plt.xlabel('Número de Menciones', fontsize=12)
    plt.ylabel('Institución', fontsize=12)
    plt.tight_layout()
    plt.savefig(CARPETA_GRAFICOS / 'top_instituciones.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(" Gráfico guardado: top_instituciones.png")
else:
    print(" No hay suficientes instituciones para graficar")

In [ ]:
# Gráfico 3: Distribución por tipo de entidad
plt.figure(figsize=(10, 6))
df['tipo'].value_counts().plot(kind='bar', color='coral')
plt.title('Distribución de Entidades por Tipo', fontsize=16, fontweight='bold')
plt.xlabel('Tipo de Entidad', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(CARPETA_GRAFICOS / 'distribucion_tipos.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado: distribucion_tipos.png")

## 📝 PASO 9: Conclusiones y Próximos Pasos

### ✅ Lo que logramos:

1. **Descarga automática** de boletines del Diario El Peruano
2. **Extracción de texto** desde PDFs
3. **Identificación de entidades** usando NER (spaCy)
4. **Base de datos estructurada** en CSV
5. **Visualizaciones** de tendencias temporales

### 🎯 Próximos pasos sugeridos:

1. **Mejorar NER**: Entrenar modelo personalizado con más ejemplos
2. **Análisis más profundo**: 
   - Correlaciones entre instituciones
   - Patrones estacionales
   - Análisis geográfico
3. **Clasificación**: Categorizar avisos por tipo (remate, disolución, etc.)
4. **Dashboard interactivo**: Usar Streamlit o Dash
5. **API de consulta**: Crear servicio REST

### 📊 Métricas del proyecto:

- ✅ Boletines procesados: [COMPLETAR]
- ✅ Entidades extraídas: [COMPLETAR]
- ✅ Instituciones identificadas: [COMPLETAR]
- ✅ Período analizado: [COMPLETAR]

## 💾 PASO 10: Exportar Resultados

Exporta todos tus resultados para el informe final:

In [ ]:
# Crear resumen para el informe
resumen = {
    'periodo': f"{df['fecha'].min().date()} a {df['fecha'].max().date()}",
    'total_boletines': len(pdfs),
    'total_entidades': len(df),
    'dias_analizados': df['fecha'].nunique(),
    'tipos_entidades': df['tipo'].value_counts().to_dict(),
    'top_10_instituciones': df[df['tipo']=='ORG']['texto'].value_counts().head(10).to_dict() if len(df[df['tipo']=='ORG']) > 0 else {}
}

# Guardar resumen
with open('resumen_proyecto.json', 'w', encoding='utf-8') as f:
    json.dump(resumen, f, ensure_ascii=False, indent=2, default=str)

# Exportar a Excel
with pd.ExcelWriter('datos_finales.xlsx') as writer:
    df.to_excel(writer, sheet_name='Todas_Entidades', index=False)
    if len(df[df['tipo']=='ORG']) > 0:
        df[df['tipo']=='ORG'].to_excel(writer, sheet_name='Instituciones', index=False)
    if len(df[df['tipo']=='LOC']) > 0:
        df[df['tipo']=='LOC'].to_excel(writer, sheet_name='Ubicaciones', index=False)

print("\n✅ EXPORTACIÓN COMPLETADA")
print("\n📁 Archivos generados:")
print("   - entidades_extraidas.csv")
print("   - entidades_extraidas.json")
print("   - datos_finales.xlsx")
print("   - resumen_proyecto.json")
print(f"   - graficos/ (3 archivos PNG)")
print("\n🎉 ¡PROYECTO COMPLETADO!")